This notebook takes a look at the scrapped results from the BoardDocs website and gets the correct addresses for each row.

Input:
- `prelim_results.csv`

In [823]:
import pandas as pd

df = pd.read_csv("prelim_results.csv")
df.head()

,URL,title_1,title_2,home_website
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,Board Policies and Guidelines,St. Joseph Public Schools,https://www.sjschools.org/
1,https://go.boarddocs.com/pa/cali/Board.nsf/Public,School Board Policy Manual,NaN,www.calsd.org
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",http://www.mapleschools.com
3,https://go.boarddocs.com/oh/rlsd/Board.nsf/Public,"585 Riverside Drive | Painesville, Ohio 44077 | 440.352.0668 | f 440.639.1959",Riverside Local School District,https://www.riversidelocalschools.com/
4,https://go.boarddocs.com/pa/shun/Board.nsf/Public,School Board Policy Manual,Southern Huntingdon County School District,http://www.shcsd.org


In [824]:
# we remove redundant info
df.loc[df["title_1"]==df["title_2"], "title_1"] = None

In [825]:
# trim whitespace
df.loc[:, "title_1"] = df["title_1"].str.strip()
df.loc[:, "title_2"] = df["title_2"].str.strip()

In [826]:
# consider the 5-digit zip code approach
# check which rows have 5-digit codes in Title1, and those with them in Title2.
# Hopefully this will be a partition

# Define a regex pattern to match a 5-digit zip code
zip_code_pattern = r'\b\d{5}\b'

# Find rows where Title1 has a 5-digit zip code
df['Title1_has_zipcode'] = df['title_1'].str.contains(zip_code_pattern, na=False)

# Find rows where Title2 has a 5-digit zip code
df['Title2_has_zipcode'] = df['title_2'].str.contains(zip_code_pattern, na=False)

In [827]:
# check if it is a partition
# first check if they add up

import numpy as np

print(f"Num of rows where title 1 has zipcode {df['Title1_has_zipcode'].sum()}")
print(f"Num of rows where title 2 has zipcode {df['Title2_has_zipcode'].sum()}")
print(f"Num of rows where title 1 or title 2 has zipcode {np.sum(df['Title1_has_zipcode'] | df['Title2_has_zipcode'])}")
print(f"Num of rows where title 1 and title 2 have zipcode {np.sum(df['Title1_has_zipcode'] & df['Title2_has_zipcode'])}")
print(f"Num of total rows {df.shape[0]}")

Num of rows where title 1 has zipcode 1440
Num of rows where title 2 has zipcode 1442
Num of rows where title 1 or title 2 has zipcode 2882
Num of rows where title 1 and title 2 have zipcode 0
Num of total rows 3906


In [828]:
# ok great, there are no rows where you can find zipcodes on both cols
# but some rows don't have zipcodes in either
# let's check them out

no_zipcode_df = df[~(df['Title1_has_zipcode'] | df['Title2_has_zipcode'])]
print(f"Num of rows without zipcode {no_zipcode_df.shape[0]}")
no_zipcode_df.sample(5)

Num of rows without zipcode 1024


,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
2248,https://go.boarddocs.com/pa/cyorksd/Board.nsf/Public,Central York School District,Board of Directors,http://www.cysd.k12.pa.us,False,False
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,False,False
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,Wakefield-Marenisco School District,(906) 224-7211,http://www.wmschools.org,False,False
2046,https://go.boarddocs.com/pa/wilsonarea/Board.nsf/Public,Wilson Area School District,Board Policy Manual,http://www.wilsonareasd.org,False,False
2604,https://go.boarddocs.com/pa/nesn/Board.nsf/Public,Policy Manual,Northeastern York School District,http://www.nesd.k12.pa.us/site/default.aspx?PageID=1,False,False


In [829]:
# let's get the proportions


print(f"Percentage of rows where title 1 has zipcode {df['Title1_has_zipcode'].sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 2 has zipcode {df['Title2_has_zipcode'].sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 1 or title 2 has zipcode {np.sum(df['Title1_has_zipcode'] | df['Title2_has_zipcode']).sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 1 and title 2 have zipcode {np.sum(df['Title1_has_zipcode'] & df['Title2_has_zipcode']).sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows with no zipcodes {no_zipcode_df.shape[0]/df.shape[0]*100:.2f}%")


Percentage of rows where title 1 has zipcode 36.87%
Percentage of rows where title 2 has zipcode 36.92%
Percentage of rows where title 1 or title 2 has zipcode 73.78%
Percentage of rows where title 1 and title 2 have zipcode 0.00%
Percentage of rows with no zipcodes 26.22%


In [830]:
# some NaNs, some "Policy Manual", "School Board Policy and Guidelines", etc
# let's check the most common values

no_zipcode_df["title_1"].value_counts().head()

title_1
School Board Policy Manual              115
Policy Manual                            54
School Board Policies and Guidelines     41
School Board Policies                    22
BoardDocs PL                              9
Name: count, dtype: int64

In [831]:
no_zipcode_df["title_2"].value_counts().head()

title_2
Board Policies                 24
                               22
School Board Policies          17
Board of Education             11
Board of Education Policies    11
Name: count, dtype: int64

In [832]:
# ok, now let's check the website col
sum(df["home_website"].isna())

53

In [833]:
# ok, unfortunately there are boarddocs without the home website linked
print(f"Percent of websites without links to official: {sum(df["home_website"].isna())/df.shape[0]*100:.3}%")

Percent of websites without links to official: 1.36%


In [834]:
# but thankfully this number is small
# let's take a look at these websites

# do not truncate the col values in display
pd.set_option('display.max_colwidth', None)

df[df["home_website"].isna()].head()

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
232,https://go.boarddocs.com/pa/camb/Board.nsf/Public,NaN,NaN,NaN,False,False
443,https://go.boarddocs.com/oh/oakhil/Board.nsf/Public,NaN,NaN,NaN,False,False
465,https://go.boarddocs.com/oh/warrenoh/Board.nsf/Public,NaN,NaN,NaN,False,False
468,https://go.boarddocs.com/pa/marp/Board.nsf/Public,School Board Policy Manual,Marple Newtown School District,NaN,False,False
533,https://go.boarddocs.com/oh/meigs/Board.nsf/Public,NaN,NaN,NaN,False,False


In [835]:
# after inspecting a few, it seems like they will usually write their school district as the h1 tag at least.

In [836]:
# back to the address
# an observation is that I don't think those that don't contain zip codes will have addresses on the website
# let's check if a single number exists in them

number_pattern = r'\d'
print("Number of no-zipcode rows that contain a number")
sum(no_zipcode_df["title_1"].str.contains(number_pattern, na=False))

Number of no-zipcode rows that contain a number


46

In [837]:
no_zipcode_df[no_zipcode_df["title_1"].str.contains(number_pattern, na=False)].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
2808,https://go.boarddocs.com/wi/ashland/Board.nsf/Public,"District Office - 2000 Beaser Avenue; Ashland, WI; (715) 682-7080",School District of Ashland Board of Education,http://www.ashland.k12.wi.us/,False,False
106,https://go.boarddocs.com/wi/edsd/Board.nsf/Public,Edgar School District 203 East Birch Street 715-352-2351,Edgar Excellence,www.edgar.k12.wi.us,False,False
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",Addison School District 4,http://www.asd4.org,False,False
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,False,False
1576,https://go.boarddocs.com/vsba/fairfax/Board.nsf/Public,"8115 Gatehouse Road, Suite 5400 | Falls Church, VA | 571-423-1075",Fairfax County School Board,http://www.fcps.edu,False,False


In [838]:
no_zipcode_df[no_zipcode_df["title_2"].str.contains(number_pattern, na=False)].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
2779,https://go.boarddocs.com/pa/iu12/Board.nsf/Public,Board Policy Manual,Lincoln Intermediate Unit 12,https://www.iu12.org/Page/9,False,False
2570,https://go.boarddocs.com/il/ccsd146/Board.nsf/Public,CCSD 146,Community Consolidated School District #146,http://www.district146.org,False,False
2936,https://go.boarddocs.com/in/scsd1/Board.nsf/Public,School Board Policies and Guidelines,Scott County School District #1,http://www.scsd1.com,False,False
1614,https://go.boarddocs.com/pa/sall/Board.nsf/Public,South Allegheny School District,2743 Washington Boulevard,http://www.southallegheny.org,False,False
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,False,False


In [839]:
# it turns out that they could either be an address with a missing zip code, or phone numbers
# there are also misc cases

# let's get a conservative (high) bound on the number of addresses that we will miss

num_no_zipcode_with_number = no_zipcode_df[no_zipcode_df["title_1"].str.contains(number_pattern, na=False) | no_zipcode_df["title_2"].str.contains(number_pattern, na=False)].shape[0]
num_no_zipcode_with_number

105

In [840]:
num_zipcode = df.shape[0] - no_zipcode_df.shape[0]

In [841]:
print(f"Worst case proportion of addresses that we will miss {num_no_zipcode_with_number/(num_no_zipcode_with_number+num_zipcode)*100:.2}%")

Worst case proportion of addresses that we will miss 3.5%


In [842]:
# we put the ones we know are correct at a new address field

df["address"] = None
df.loc[df['Title1_has_zipcode'], "address"] = df["title_1"]
df.loc[df['Title2_has_zipcode'], "address"] = df["title_2"]
df.sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address
1772,https://go.boarddocs.com/ny/nhpgcp/Board.nsf/Public,"1950 Hillside Avenue | New Hyde Park, NY 11040 | (516) 434-2300",New Hyde Park-Garden City Park Union Free School District,http://www.nhp-gcp.org/,True,False,"1950 Hillside Avenue | New Hyde Park, NY 11040 | (516) 434-2300"
3494,https://go.boarddocs.com/mi/cal/Board.nsf/Public,Caledonia Community Schools,Neola Board of Education Policy Manual,http://www.calschools.org,False,False,None
178,https://go.boarddocs.com/mo/wsdr4/Board.nsf/Public,"280 Interstate Drive | Wentzville, Missouri 63385 | 636-327-3800 | f - 636-327-8611",Wentzville R-IV School District,http://wentzville.k12.mo.us/,True,False,"280 Interstate Drive | Wentzville, Missouri 63385 | 636-327-3800 | f - 636-327-8611"
1108,https://go.boarddocs.com/mi/holtp/Board.nsf/Public,Holt Public Schools,Board of Education Policies and Administrative Guidelines,www.hpsk12.net,False,False,None
943,https://go.boarddocs.com/vsba/amelia/Board.nsf/Public,"8701 Otterburn Road | Suite 101 | Amelia, Virginia 23002 | 804-561-2621",Amelia County Public Schools,http://www.amelia.k12.va.us/index.htm,True,False,"8701 Otterburn Road | Suite 101 | Amelia, Virginia 23002 | 804-561-2621"


In [843]:
# now, let's try to get the school district name.
# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

In [844]:
# number of rows with schools in at least one col
df[df["title_1_has_school"] | df["title_2_has_school"]].shape[0]

3206

In [845]:
# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

356

In [846]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
1118,https://go.boarddocs.com/pa/mntr/Board.nsf/Public,School Board Policy Manual,Montrose Area School District,http://www.masd.info,False,False,None,True,True
3550,https://go.boarddocs.com/pa/hazl/Board.nsf/Public,School Board Policy Manual,Hazleton Area School District,http://www.hasdk12.org,False,False,None,True,True
2323,https://go.boarddocs.com/pa/bntw/Board.nsf/Public,"School Board meetings, agendas and policies",Bentworth School District,http://www.bentworth.org/,False,False,None,True,True
243,https://go.boarddocs.com/mi/burt/Board.nsf/Public,School Board Policies and Guidelines,Burt Township School District,www.burt.school,False,False,None,True,True
2033,https://go.boarddocs.com/mo/osage/Board.nsf/Public,"1501 School Road| Lake Ozark, MO 65049 | (573) 365-4091",School of the Osage,https://osageschools.org/,True,False,"1501 School Road| Lake Ozark, MO 65049 | (573) 365-4091",True,True


In [847]:
# there is boilerplate text like School Board Policies
# let's try to get the top few popular ones and remove them

df["title_1"].value_counts().head(10)

title_1
School Board Policy Manual              118
Policy Manual                            55
School Board Policies and Guidelines     41
School Board Policies                    22
BoardDocs PL                              9
Board Policy and Guidelines               8
Board Policies                            7
School Board Policies and Bylaws          7
School Board Policies & Bylaws            6
Board Policies and Guidelines             6
Name: count, dtype: int64

In [848]:
df["title_2"].value_counts().head(10)

title_2
Board Policies                      25
                                    25
School Board Policies               18
Board of Education                  14
Board of Education Policies         12
Board of Education Policy Manual    10
NEOLA Board Policies                10
Policy Manual                        6
School Board Policy Manual           6
Board Policy Manual                  5
Name: count, dtype: int64

In [849]:
# let's remove these

remove_title_1_list = df["title_1"].value_counts().head(10).index
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
Board Policy                          6
School District Policies              6
Board Policy Manual                   5
Board Policies and Bylaws             5
School Board Policies & Guidelines    4
eGovernance Site                      4
School Board Policy                   4
School District Policy Manual         3
Board Policies & Bylaws               3
Board Policy and Bylaws               3
Name: count, dtype: int64

In [850]:
# let's remove these
remove_title_1_list = df["title_1"].value_counts().head(10).index
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
Board of Education Policies and Guidelines                                                  3
School Board Policy and Guidelines                                                          3
School Board Policies and Administrative Guidelines                                         3
                                                                                            3
Board of Education Policies                                                                 3
2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180    2
Bloomfield School District                                                                  2
1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500                           2
School Board Policy & Guidelines                                                            2
1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009           2
Name: count, dtype: int64

In [851]:
# let's remove the first few
remove_title_1_list = df["title_1"].value_counts().head(5).index
remove_title_1_list

Index(['Board of Education Policies and Guidelines',
       'School Board Policy and Guidelines',
       'School Board Policies and Administrative Guidelines', '',
       'Board of Education Policies'],
      dtype='object', name='title_1')

In [852]:
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180    2
1300 Sherman Street Ste 222 | Sturgis, SD  57785                                            2
Bloomfield School District                                                                  2
1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500                           2
School Board Policy & Guidelines                                                            2
1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009           2
Electronic Governance System                                                                2
Policy Manual and Agendas                                                                   2
Board of Education                                                                          2
60 Jefferson Street, Suite 3• Monticello, NY 12701 • 845-794-7700                           2
Name: count, dtype: int64

In [853]:
# pick those
remove_title_1_list = [
    "School Board By-Laws and Policies",
    "School Board policies and guidelines",
    "Policy Manual and Agendas",
    "Board of Education Policies",
    "School Board Policy and Bylaws",
    "Board Policy and Bylaws",
    "Board Policies and By-Laws",
    "School Board Policies ",
    "Board of Education",
    "School Board Policies and Guidlines",
    "School Board Bylaws and Policies",
    "SCHOOL BOARD POLICIES",
    "Board Policies & Guidelines",
    "School Board Policy & Guidelines",
    "Electronic Governance System",
    "School Board Agendas and Minutes",
    "School Board Policy and Administrative Guidelines",
    "School District Policies / Policy Manual",
    "School Policy Manual",
    "School Board Meetings",
    "School Board Policies & Ad Guidelines",
    "School Board meetings, agendas and policies",
    "School Board Policies and By Laws",
    "School board policies and manuals",
    "Board of School Directors",
    "School Board Agendas, Minutes, Policies and Guidelines",
    "Board Approved School Policy",
    "School Board Policies & Administrative Guidelines",
    "School Board Policy",
    "School Board Agendas, Policies and Guidelines",
    "School Board Bylaws/Policies/Guidelines",
    "SchoolBoard Policy Manual",
    "School Board Policy and Guidlines",
    "School Board Policy",
    "Governing Board Policy",
    "Board policy and guidelines",
    "Policies & By Laws",
    "Board of Trustees Policies and Administrative Guidelines",
    "Board of Education Policy and Bylaws"
    
]
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
print(df["title_1"].value_counts().index[:10])

Index(['2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180',
       '200 Reid Street | Palatka, FL 32177 | (386) 329-0602',
       '60 Jefferson Street, Suite 3• Monticello, NY 12701 • 845-794-7700',
       'Bloomfield School District',
       '1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500',
       '1300 Sherman Street Ste 222 | Sturgis, SD  57785',
       '315 N. French Avenue | Arlington  WA 98223 | 360.618.6200 | f 360.618.6221',
       '1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009',
       'Cleveland Metropolitan School District',
       '6301 Springside Avenue | Downers Grove, IL 60516 | Ph: (630) 795-7100  | Fx: (630) 795-7199'],
      dtype='object', name='title_1')


In [854]:
# do this for title_2
# let's remove these
# NOTE: NEOLOA Board Policies might provide coarse information on whether the school could be located

remove_title_2_list = df["title_2"].value_counts().head(10).index
remove_title_2_list

Index(['Board Policies', '', 'School Board Policies', 'Board of Education',
       'Board of Education Policies', 'Board of Education Policy Manual',
       'NEOLA Board Policies', 'Policy Manual', 'School Board Policy Manual',
       'Board Policy Manual'],
      dtype='object', name='title_2')

In [855]:
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().head(10)

title_2
eGovernance Site                 4
Bylaws & Policies                4
Board of Education Policy        4
Board of Education Meetings      3
Board Policy                     3
Neola Board Policies             3
Board of Directors               3
Arlington Public Schools         3
Putnam County School District    2
BoardDocs - Meeting Agendas      2
Name: count, dtype: int64

In [856]:
remove_title_2_list = df["title_2"].value_counts().head(4).index
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().head(10)

title_2
Arlington Public Schools                  3
Neola Board Policies                      3
Board of Directors                        3
Board Policy                              3
Board Agendas                             2
Monticello Central School District        2
Meetings, Agendas and Information         2
Green Local Schools                       2
Santa Clara County Office of Education    2
Meade County, South Dakota                2
Name: count, dtype: int64

In [857]:
remove_title_2_list = [
    "Meetings and Information",
    "Meetings, Agendas and Information",
    "Board Policy Manual",
    "Board of Education Meetings",
    "Board of Directors",
    "Policies",
    "BoardDocs - Meeting Agendas",
    "Board of Education Policies ",
    'Board Agendas',
    "NEOLA Board Policy",
    'NEOLA Board of Education Policies',
    'Neola Board Policies', 
    'NEOLA Board of Education Policy Manual',
    'NEOLA Policies', 
    'Meetings, Agendas, Information',
    'School Board Policies and Guidelines',
    "School Board Policies and Bylaws",
    "NEOLA School Board Policies",
    "Board of School Trustees Policy Manual",
    "School Board Policy	",
    "School Board Policies & Administrative Regulations	",
    "School Board Policy",
    "Board of School Trustees",
    "SCHOOL POLICIES AND GUIDELINES"
]
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().index[:20]

Index(['Arlington Public Schools', 'Board Policy',
       'Central Valley School District', 'Meade County, South Dakota',
       'Sumter District Schools', 'Santa Clara County Office of Education',
       'Green Local Schools', 'eGovernance System',
       'Monticello Central School District',
       'Community High School District 99', 'Putnam County School District',
       '8485 Homestead, Zeeland, MI 49464 Phone: 616-748-5637',
       'Iowa City Community School District',
       '2045 School Street North Collins, NY 14111', 'Barry ISD',
       'Adena Local Schools',
       '801 Corporate Centre Drive | O'Fallon, MO 63368 | Phone: 636-851-4000',
       '1323 E. 7th Street, Lockport, IL 60441',
       'Phone: 330-627-2181, - Fax: 330-627-2182',
       'Johnson County School District #1'],
      dtype='object', name='title_2')

In [858]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

89

In [859]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
429,https://go.boarddocs.com/il/olchs/Board.nsf/Public,"9400 Southwest Hwy. Oak Lawn, IL 60453 Phone: (708) 424-5200 District Fax: (708) 424-5297 School Fax: (708) 424-5263",Oak Lawn Community High School,http://www.olchs.org/,True,False,"9400 Southwest Hwy. Oak Lawn, IL 60453 Phone: (708) 424-5200 District Fax: (708) 424-5297 School Fax: (708) 424-5263",True,True
2603,https://go.boarddocs.com/az/ftusd/Board.nsf/Public,"15560 W Elementary School Road, Fort Thomas, AZ 85536 | Phone 928-485-9423 | Fax 928-485-3019",Fort Thomas Unified School District,http://www.ftusd.org/,True,False,"15560 W Elementary School Road, Fort Thomas, AZ 85536 | Phone 928-485-9423 | Fax 928-485-3019",True,True
3629,https://go.boarddocs.com/ny/schsd/Board.nsf/Public,"Sewanhaka High School, Floral Park, NY 11001",Sewanhaka Central High School District,http://www.sewanhakaschools.org,True,False,"Sewanhaka High School, Floral Park, NY 11001",True,True
701,https://go.boarddocs.com/ny/letchworth/Board.nsf/Public,Letchworth Central School District,"5550 School Road Gainesville, NY 14066",https://www.letchworth.k12.ny.us/,False,True,"5550 School Road Gainesville, NY 14066",True,True
3565,https://go.boarddocs.com/nh/rivendell/Board.nsf/Public,Rivendell School District,PO Box 271 10 School Drive Orford NH 03777,https://www.rivendellschool.org/,False,True,PO Box 271 10 School Drive Orford NH 03777,True,True


In [860]:
# I asked ChatGPT to inspect and find more titles I can remove

remove_title_1_list = [
    "School Board Agendas, Minutes, and Policies",
    "School Board Policies, Bylaws, and Guidelines",
    "School Policies and Guidelines",
    "School Board Policy Manual",
    "Board of School Trustees Policy",
    "School Board Agendas and Policies",
    "SchoolBoard Policy Manual",
    "School Board Agendas, Policies, Rules and Exhibits",
    "School Board Policy",
    "SchoolBoard Policy Manual ",
    "School District Policies ",
    "Board Meetings and Policies",
    "Board Management System",
    "Board of Education Policy and Guidelines",
    "Board Policy and By-Laws",
    "Policy and Bylaws",
    "Board Bylaws & Policies",
    "Bylaws & Policies",
    "Board of Trustees Policies",
    "Policies - Bylaws",
    "Bylaws and Policies",
    "NEOLA Policies",
    "Board of Education Bylaws and Policies",
    "School Board Policy Manual",
    "Board Administrative Guidelines , Bylaws, Forms and Policies",
    "Board Policies, Administrative Guidelines, and Forms",
    "Board Policies, Bylaws, Administrative Guidelines, Forms",
    "Board Agendas and Policies",
    "Policies of the Board of Education",
    "School Board Policies and Guidelines",
    "Corporation Board Policies and Guidelines",
    "NEOLA Board Policies and By-Laws",
    "Policies & Bylaws",
    "Board of Education Policies and Guidelines",
    "Policies - Bylaws",
    "Policies of the Board of Education",
    "Board Policy Handbook",
    "Board of Education Policies",
    "Board Bylaws and Policies",
    "Board Policy Manual and Administrative Guidelines",
    "Board Policies and Administrative Guidelines",
    "Board of Education NEOLA Policy",
    "Board of Education Policies and Administrative Guidelines",
    "Board of School Trustees Policy Manual",
    "School Board Policies and Guidelines",
    "Board Policy",
    "Policies and Administrative Guidelines",
    "Board of Education Policy and Administrative Guidelines",
    "Policies & Administrative Guidelines",
    "Meetings, Agendas, Policy Manual",
    "Joint Operating Committee (JOC) Policy Manual",
    "Board Policy & Guidelines",
    "BOARD OF TRUSTEES",
    "School Board Policies, Meeting Agendas and Minutes",
    "Board Policies and By-laws",
    "Board of Regents",
    "Board of Education Policies and By-Laws",
    "Board of Education Bylaws and Policies/Administrative Guidelines",
    "BoardDocs LT",
    "Fairless District Policy Manual",
    "Board of Education Bylaws & Policies",
    "Board Policies, Administrative Guidelines and Forms",
    "Board Policy & Bylaws",
    "Success for all in the 21st Century . . ."
]

df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None

In [861]:
# I asked ChatGPT to inspect and find more titles I can remove

remove_title_2_list = [
    "POLICY",
    "Neola Board Policies & Guidelines",
    "Board of Education Policy and Guidelines",
    "NEOLA Policy Manual",
    "Board Policy",
    "Board Policies and Guidelines",
    "Policies And Administrative Guidelines",
    "Board Policies, Administrative Guidelines, and Forms",
    "Board of Education Policy",
    "Board Bylaws and Policies",
    "Policies - Bylaws",
    "Board of Education Bylaws and Policies",
    "Neola Board Policy & Administrative Guidelines",
    "Board Policy Handbook",
    "Policies of the Board of Education",
    "NEOLA Board Policy Manual",
    "Policies & Administrative Guidelines",
    "Policies & Bylaws",
    "Board Bylaws and Policies",
    "Board Policies",
    "Board of Education Policies",
    "Board of Education Policy and Administrative Guidelines",
    "Board Policy Manual",
    "Board of Education Policies and Administrative Guidelines",
    "BoardDocs LT",
    "Policies - Bylaws",
    "Policies of the Board of Education",
    "Board Policy and By-Laws",
    "Policies and Administrative Guidelines",
    "Board of Education Bylaws & Policies",
    "Board Policy",
    "Policies & Administrative Guidelines",
    "Board of Education School Policies",
    "School Board",
    "School Board Policies & Administrative Regulations",
    "School Board Policy",
    "Board of School Trustees Policy Manual",
    "School Board Policies and Guidelines",
    "School Board Policy ",
    "Board Policy",
    "Bylaws and Policies",
    "Meetings, Agenda and Information",
    "eGovernance System",
    "Meeting Packets",
    "Opportunity. Equity. Social Justice.",
    "Neola Board Policies & Guidelines",
    "Board Of Education Policy",
    "Board of Education NEOLA Policy",
    "Board Of Education Policies",
    "NEOLA - Board of Education Policies",
    "Board Policy Manual and Administrative Guidelines",
    "Policies & By-Laws",
    "Providing today's students opportunities to become tomorrow's leaders",
    "Board of School Trustees Policy",
    "Corporation Policies",
    "Board of Education Neola Polcies",
    "NEOLA Board Policies and By-Laws",
    "Board of Education Policy /Administrative Guidelines",
    "Board of Education Policies and Bylaws",
    "Board of Education Policies and Guidelines",
    "Neola Board of Education Policy Manual",
    "Neola - Board Policy",
    "Neola Board Policy",
    "Creating the Greatest Opportunities for Our Students"
]

df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None

In [862]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

79

In [863]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
3549,https://go.boarddocs.com/ny/wvalley/Board.nsf/Public,West Valley Central School District,"5359 School Street West Valley, NY 14171",https://www.wvalley.org/,False,True,"5359 School Street West Valley, NY 14171",True,True
3124,https://go.boarddocs.com/pa/nleb/Board.nsf/Public,346 School Drive,Northern Lebanon School District,www.norleb.org,False,False,None,True,True
3281,https://go.boarddocs.com/pa/naza/Board.nsf/Public,NASD Board of School Directors Meeting,Nazareth Area School District,http://www.nazarethasd.org,False,False,None,True,True
659,https://go.boarddocs.com/ny/willsborocsd/Board.nsf/Public,Willisboro Central School District,29 School Lane | Willsboro NY 12996 | (518) 963-4456,http://www.willsborocsd.org/,False,True,29 School Lane | Willsboro NY 12996 | (518) 963-4456,True,True
1144,https://go.boarddocs.com/pa/free/Board.nsf/Public,"1702 School Street | Freedom, PA 15042 | (724) 775-5464",Freedom Area School District,http://www.freedomareaschools.org,True,False,"1702 School Street | Freedom, PA 15042 | (724) 775-5464",True,True


In [864]:
# since a lot of it is from addresses,
# I will remove those where we have ported to the address field

df.loc[df["title_1"] == df["address"], "title_1"] = None
df.loc[df["title_2"] == df["address"], "title_2"] = None

In [865]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

20

In [866]:
# from 97 to 38, pretty good

# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
1022,https://go.boarddocs.com/wi/tlsd/Board.nsf/Public,Twin Lakes School District #4,Lakewood School Board Policies,http://www.twinlakes.k12.wi.us,False,False,None,True,True
2366,https://go.boarddocs.com/oh/naps/Board.nsf/Public,www.napoleonareaschools.org | 419-599-7015,Napoleon Area City School District,www.napoleon.k12.oh.us,False,False,None,True,True
3281,https://go.boarddocs.com/pa/naza/Board.nsf/Public,NASD Board of School Directors Meeting,Nazareth Area School District,http://www.nazarethasd.org,False,False,None,True,True
3124,https://go.boarddocs.com/pa/nleb/Board.nsf/Public,346 School Drive,Northern Lebanon School District,www.norleb.org,False,False,None,True,True
2031,https://go.boarddocs.com/wi/vasd/Board.nsf/Public,Verona Area School District,Verona Area School District,http://www.verona.k12.wi.us,False,False,None,True,True


In [867]:
# now, let's check those with both titles still intact

n = df[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]
print(f"Number of rows with both cols intact: {n}")

Number of rows with both cols intact: 156


In [868]:
# let's make a school district column
# only do those to rows where EXACTLY one column has the word school district

df["title_1_has_school_district"] = df["title_1"].str.contains("school district",case=False,na=False)
df["title_2_has_school_district"] = df["title_2"].str.contains("school district",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]].shape[0]

4

In [869]:
# check out these with both
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]]

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school,title_1_has_school_district,title_2_has_school_district
538,https://go.boarddocs.com/ny/trivc/Board.nsf/Public,Tri-Valley Central School District Board of Education,Tri-Valley Central School District,http://www.trivalleycsd.org/,False,False,None,True,True,True,True
769,https://go.boarddocs.com/pa/epns/Board.nsf/Public,EAST PENNSBORO AREA SCHOOL DISTRICT,East Pennsboro Area School District,http://www.epasd.org,False,False,None,True,True,True,True
1593,https://go.boarddocs.com/oh/labr/Board.nsf/Public,Policies of the LaBrae Local School District,LaBrae Local School District,https://labrae.school,False,False,None,True,True,True,True
2031,https://go.boarddocs.com/wi/vasd/Board.nsf/Public,Verona Area School District,Verona Area School District,http://www.verona.k12.wi.us,False,False,None,True,True,True,True


In [870]:
# we can remove all in title_1
df.loc[df["title_1_has_school_district"] & df["title_2_has_school_district"], "title_1"] = None

In [871]:
# let's make a school district column
# only do those to rows where EXACTLY one column has the word school district

df["title_1_has_school_district"] = df["title_1"].str.contains("school district",case=False,na=False)
df["title_2_has_school_district"] = df["title_2"].str.contains("school district",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]].shape[0]

0

In [872]:
# now we can assign the school_district column

df.loc[df["title_1_has_school_district"],"school_district"] = df["title_1"]
df.loc[df["title_1_has_school_district"],"title_1"] = None

df.loc[df["title_2_has_school_district"],"school_district"] = df["title_2"]
df.loc[df["title_2_has_school_district"],"title_2"] = None

In [873]:
# now let's look at the remaining columns
# first check those where both cols are still intact

df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]

70

In [874]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school,title_1_has_school_district,title_2_has_school_district,school_district
3356,https://go.boarddocs.com/wa/cowa/Board.nsf/Public,City Council Chambers ~ 500 E. Main Street ~ 509-488-5686 ~ www.othellowa.gov,Othello Washington ~ City Council ~ Serving The Community,http://www.othellowa.gov,False,False,None,False,False,False,False,NaN
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,False,False,None,True,False,False,False,NaN
2108,https://go.boarddocs.com/oh/maplecc/Board.nsf/Public,SCHOOL POLICIES AND GUIDELINES,MAPLEWOOD CAREER CENTER,http://www.mwood.cc/,False,False,None,True,False,False,False,NaN
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,False,False,None,False,True,False,False,NaN
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,False,False,None,False,True,False,False,NaN


In [875]:
# now, let's check those with policy or policies in their titles

df["title_1_has_policy"] = df["title_1"].str.contains("policy",case=False,na=False) | df["title_1"].str.contains("policies",case=False,na=False)
df["title_2_has_policy"] = df["title_2"].str.contains("policy",case=False,na=False) | df["title_2"].str.contains("policies",case=False,na=False)

# number of rows with policy in any cols
df[df["title_1_has_policy"] | df["title_2_has_policy"]].shape[0]

50

In [876]:
# declutter columns
df = df.loc[:,["URL","title_1","title_2","home_website","address","school_district"]]

In [877]:
df.head()

,URL,title_1,title_2,home_website,address,school_district
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,None,St. Joseph Public Schools,https://www.sjschools.org/,None,NaN
1,https://go.boarddocs.com/pa/cali/Board.nsf/Public,None,NaN,www.calsd.org,None,NaN
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,None,http://www.mapleschools.com,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",NaN
3,https://go.boarddocs.com/oh/rlsd/Board.nsf/Public,None,None,https://www.riversidelocalschools.com/,"585 Riverside Drive | Painesville, Ohio 44077 | 440.352.0668 | f 440.639.1959",Riverside Local School District
4,https://go.boarddocs.com/pa/shun/Board.nsf/Public,None,None,http://www.shcsd.org,None,Southern Huntingdon County School District


In [878]:
# let's try to common title_1 and title_2
# first standardize None and NaN

print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_1"])
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_1"].isna())

1    None
Name: title_1, dtype: object
1    True
Name: title_1, dtype: bool


In [879]:
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_2"])
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_2"].isna())

1    NaN
Name: title_2, dtype: object
1    True
Name: title_2, dtype: bool


In [880]:
# set them to None
df.loc[df["title_1"].isna(), "title_1"] = None
df.loc[df["title_2"].isna(), "title_2"] = None

In [881]:
# check those with both not None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]

70

In [882]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].sample(5)

,URL,title_1,title_2,home_website,address,school_district
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN
3164,https://go.boarddocs.com/mi/clio/Board.nsf/Public,Clio Area Schools,Home of the Mustangs,http://www.clioschools.org,None,NaN


In [883]:
# get those with phone numbers out
import re

# Function to check if a string contains a phone number
def contains_phone_number(value):
    # Regular expression for phone numbers
    phone_pattern = re.compile(r'\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
    if pd.isnull(value):
        return False
    return bool(phone_pattern.search(str(value)))

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df['contains_phone_number_title_2'] = df['title_2'].apply(contains_phone_number)

In [884]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_1"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
1112,https://go.boarddocs.com/la/pcpsb/Board.nsf/Public,"337 Napoleon Street ● New Roads, Louisiana ● p 225-638-8674 ● f 225-638-3237",Pointe Coupee Parish School System,http://www.pcpsb.net/,None,NaN,True,False
1576,https://go.boarddocs.com/vsba/fairfax/Board.nsf/Public,"8115 Gatehouse Road, Suite 5400 | Falls Church, VA | 571-423-1075",Fairfax County School Board,http://www.fcps.edu,None,NaN,True,False
1987,https://go.boarddocs.com/oh/nls/Board.nsf/Public,600 Lemoyne Rd | Northwood OH | P: (419) 691-3888 | F: (419) 697-2470,Northwood Local Schools,http://www.northwoodschools.org/site/default.aspx?PageID=1,None,NaN,True,False
2150,https://go.boarddocs.com/oh/polaris/Board.nsf/Public,"7285 Old Oak Blvd., | Middleburg Heights, OH | 440-891-7600",Polaris Career Center,http://www.polaris.edu/,None,NaN,True,False
3356,https://go.boarddocs.com/wa/cowa/Board.nsf/Public,City Council Chambers ~ 500 E. Main Street ~ 509-488-5686 ~ www.othellowa.gov,Othello Washington ~ City Council ~ Serving The Community,http://www.othellowa.gov,None,NaN,True,False


In [885]:
# all the above are address
move_these = both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_1"]]["URL"]
df.loc[df["URL"].isin(move_these), "address"] = df[df["URL"].isin(move_these)]["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

In [886]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_2"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
159,https://go.boarddocs.com/in/resc/Board.nsf/Public,Randolph Eastern School Corporation,7659644994,http://www.resc.k12.in.us,None,NaN,False,True
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN,False,True
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,None,NaN,False,True
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN,False,True
1608,https://go.boarddocs.com/oh/sclsd/Board.nsf/Public,South Central Local Schools,3305 Greenwich Angling Rd | 419-752-3815,http://www.south-central.org,None,NaN,False,True
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN,False,True
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN,False,True
2306,https://go.boarddocs.com/mi/clark/Board.nsf/Public,Clarkston Community Schools,248-623-5400,http://www.clarkston.k12.mi.us,None,NaN,False,True
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN,False,True
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,None,NaN,False,True


In [887]:
# move those two that are address
move_these = ["https://go.boarddocs.com/oh/sclsd/Board.nsf/Public", "https://go.boarddocs.com/oh/sidn/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "address"] = df["title_2"]
df.loc[df["URL"].isin(move_these), "title_2"] = None

In [888]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_2"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
159,https://go.boarddocs.com/in/resc/Board.nsf/Public,Randolph Eastern School Corporation,7659644994,http://www.resc.k12.in.us,None,NaN,False,True
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN,False,True
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,None,NaN,False,True
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN,False,True
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN,False,True
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN,False,True
2306,https://go.boarddocs.com/mi/clark/Board.nsf/Public,Clarkston Community Schools,248-623-5400,http://www.clarkston.k12.mi.us,None,NaN,False,True
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN,False,True
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,None,NaN,False,True
2683,https://go.boarddocs.com/mi/slps/Board.nsf/Public,Spring Lake Public Schools,616-846-5500,http://www.springlakeschools.org,None,NaN,False,True


In [889]:
# move the others to phone numbers
df["phone"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["contains_phone_number_title_2"], "phone"] = df["title_2"]
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["contains_phone_number_title_2"], "title_2"] = None

In [890]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,None,NaN,False,False,None
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
356,https://go.boarddocs.com/ca/mendocino/Board.nsf/Public,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",https://www.mendocino.edu,None,NaN,False,False,None
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None


In [891]:
# check those with numbers
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
2878,https://go.boarddocs.com/pa/daup/Board.nsf/Public,"School Board Policy Manual 6001 Locust Lane, Harrisburg, PA",Dauphin County Technical School,http://www.dcts.org,None,NaN,False,False,None
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [892]:
# edit the outlier
df.loc[df["URL"]=="https://go.boarddocs.com/pa/daup/Board.nsf/Public", "title_1"] = "6001 Locust Lane, Harrisburg, PA"

In [893]:
# check those with numbers
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
2878,https://go.boarddocs.com/pa/daup/Board.nsf/Public,"6001 Locust Lane, Harrisburg, PA",Dauphin County Technical School,http://www.dcts.org,None,NaN,False,False,None
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [894]:
# move these two that are addresses
move_these = ["https://go.boarddocs.com/pa/daup/Board.nsf/Public", "https://go.boarddocs.com/wi/chilsd/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "address"] = df["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

# check
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [895]:
# move the second one to district and delete the two others
df.loc[df["URL"]=="https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public", "title_1"] = None

# check
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [896]:
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False), "title_1"] = None

In [897]:
# do the same for title_2
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,None,NaN,False,False,None
356,https://go.boarddocs.com/ca/mendocino/Board.nsf/Public,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",https://www.mendocino.edu,None,NaN,False,False,None
1203,https://go.boarddocs.com/in/brownsburg/Board.nsf/Public,Brownsburg Community School Corporation,"310 Stadium Drive Brownsburg, IN",https://www.brownsburg.k12.in.us/,None,NaN,False,False,None


In [898]:
# move them to address
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False),"address"] = df["title_2"]
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False),"title_2"] = None

In [899]:
# check the rest
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx,:]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None


In [900]:
# check those with schools in one but not in the other
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_1"].str.contains("school",case=False,na=False) & ~df["title_2"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,None,NaN,False,False,None
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN,False,False,None
1980,https://go.boarddocs.com/mi/fwlv/Board.nsf/Public,Fowlerville Community Schools,Home of the Gladiators,http://www.fowlervilleschools.org,None,NaN,False,False,None
2047,https://go.boarddocs.com/ok/okcps/Board.nsf/Public,Oklahoma City Public Schools,Ignite Passion. Instill Pride.,https://go.boarddocs.com/ok/okcps/Board.nsf/,None,NaN,False,False,None
2108,https://go.boarddocs.com/oh/maplecc/Board.nsf/Public,SCHOOL POLICIES AND GUIDELINES,MAPLEWOOD CAREER CENTER,http://www.mwood.cc/,None,NaN,False,False,None
2301,https://go.boarddocs.com/vsba/louisa/Board.nsf/Public,Louisa County Public Schools,Learners' Community,http://www.lcps.k12.va.us/education/components/scrapbook/default.php?sectiondetailid=1308&PHPSESSID=00a5e9972c66fd8162a098aed2356931,None,NaN,False,False,None
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN,False,False,None
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,None,NaN,False,False,None
3126,https://go.boarddocs.com/oh/zville/Board.nsf/Public,Zanesville City Schools District,Home of the Blue Devils,http://www.zanesville.k12.oh.us,None,NaN,False,False,None


In [901]:
# take out the outlier
df.loc[df["URL"] == "https://go.boarddocs.com/oh/maplecc/Board.nsf/Public", "title_1"] = None

In [902]:
# check those with schools in one but not in the other
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_1"].str.contains("school",case=False,na=False) & ~df["title_2"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,None,NaN,False,False,None
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN,False,False,None
1980,https://go.boarddocs.com/mi/fwlv/Board.nsf/Public,Fowlerville Community Schools,Home of the Gladiators,http://www.fowlervilleschools.org,None,NaN,False,False,None
2047,https://go.boarddocs.com/ok/okcps/Board.nsf/Public,Oklahoma City Public Schools,Ignite Passion. Instill Pride.,https://go.boarddocs.com/ok/okcps/Board.nsf/,None,NaN,False,False,None
2301,https://go.boarddocs.com/vsba/louisa/Board.nsf/Public,Louisa County Public Schools,Learners' Community,http://www.lcps.k12.va.us/education/components/scrapbook/default.php?sectiondetailid=1308&PHPSESSID=00a5e9972c66fd8162a098aed2356931,None,NaN,False,False,None
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN,False,False,None
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,None,NaN,False,False,None
3126,https://go.boarddocs.com/oh/zville/Board.nsf/Public,Zanesville City Schools District,Home of the Blue Devils,http://www.zanesville.k12.oh.us,None,NaN,False,False,None
3164,https://go.boarddocs.com/mi/clio/Board.nsf/Public,Clio Area Schools,Home of the Mustangs,http://www.clioschools.org,None,NaN,False,False,None


In [903]:
# all slogans in second col, delete
df.loc[idx, "title_2"] = None

In [904]:
# do it for the second col
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_2"].str.contains("school",case=False,na=False) & ~df["title_1"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
3308,https://go.boarddocs.com/in/hses/Board.nsf/Public,For additional information click the HOUSE icon to view streamed/archived board meeting videos.,Hamilton Southeastern Schools,https://www.hseschools.org/meet-hse/board,None,NaN,False,False,None
3393,https://go.boarddocs.com/mo/nixa/Board.nsf/Public,Board of Education Meeting Information,Nixa Public Schools,http://www.nixapublicschools.net,None,NaN,False,False,None
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,None,NaN,False,False,None
3443,https://go.boarddocs.com/mi/tda/Board.nsf/Public,THE DEARBORN ACADEMY,Public Charter School,http://www.thedearbornacademy.org,None,NaN,False,False,None


In [905]:
# edit the outlier
df.loc[df["URL"] == "https://go.boarddocs.com/mi/tda/Board.nsf/Public", "title_2"] = None
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_2"].str.contains("school",case=False,na=False) & ~df["title_1"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
3308,https://go.boarddocs.com/in/hses/Board.nsf/Public,For additional information click the HOUSE icon to view streamed/archived board meeting videos.,Hamilton Southeastern Schools,https://www.hseschools.org/meet-hse/board,None,NaN,False,False,None
3393,https://go.boarddocs.com/mo/nixa/Board.nsf/Public,Board of Education Meeting Information,Nixa Public Schools,http://www.nixapublicschools.net,None,NaN,False,False,None
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,None,NaN,False,False,None


In [906]:
# delete the first col
df.loc[idx,"title_1"] = None

In [907]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None


In [908]:
# for those with the word policy, make it the first col
policy_pattern = r'polic(y|ies)'
idx = df["title_2"].str.contains(policy_pattern, case=False,na=False)
df.loc[idx, ["title_1", "title_2"]] = df.loc[idx,["title_2","title_1"]].values

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/52244175.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df["title_2"].str.contains(policy_pattern, case=False,na=False)


In [909]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,ROD Board Policy,Ripley-Ohio-Dearborn Special Education Cooperative,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None


In [910]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/2999244493.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False)]


,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,ROD Board Policy,Ripley-Ohio-Dearborn Special Education Cooperative,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
2720,https://go.boarddocs.com/mi/giresa/Board.nsf/Public,Board Policies and Bylaws,Gratiot-Isabella Regional Education Service District,http://www.giresd.net,None,NaN,False,False,None
3326,https://go.boarddocs.com/mi/trav/Board.nsf/Public,TCAPS School Board Policies & Guidelines,Traverse City Area Public Schools,http://www.tcaps.net/board,None,NaN,False,False,None
3407,https://go.boarddocs.com/oh/tructc/Board.nsf/Public,TCTC Board Policies,Trumbull Career and Technical Center,http://www.tctchome.com,None,NaN,False,False,None
3525,https://go.boarddocs.com/pa/vnang/Board.nsf/Public,Venango Technology Center Policy Manual,Venango Technology Center,http://www.vtc1.org,None,NaN,False,False,None


In [911]:
# can remove title_1 for these
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False), "title_1"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/786511925.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False), "title_1"] = None


,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
2889,https://go.boarddocs.com/ut/uen/Board.nsf/Public,Utah Education and Telehealth Network,UETN Governing Board,http://www.uetn.org,None,NaN,False,False,None
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County Schools,Jefferson County School Board,https://www.jeffersonschools.net,None,NaN,False,False,None
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN,False,False,None
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,None,NaN,False,False,None
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,None,NaN,False,False,None


In [912]:
# do the same for board
idx = df["title_2"].str.contains("board", case=False,na=False)
df.loc[idx, ["title_1", "title_2"]] = df.loc[idx,["title_2","title_1"]].values

In [913]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains("board",case=False,na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City School Board,Falls Church City Public Schools,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2889,https://go.boarddocs.com/ut/uen/Board.nsf/Public,UETN Governing Board,Utah Education and Telehealth Network,http://www.uetn.org,None,NaN,False,False,None
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County School Board,Jefferson County Schools,https://www.jeffersonschools.net,None,NaN,False,False,None
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,None,NaN,False,False,None


In [914]:
# can remove title_1 for these
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains("board",case=False,na=False), "title_1"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN,False,False,None
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,None,NaN,False,False,None


In [915]:
# just do case by case
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/ca/sjccd/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/ca/hawking/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/mi/badax/Board.nsf/Public", "title_2"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/in/fcsc/Board.nsf/Public", "title_1"] = None

df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone


In [916]:
# check those with both not None
n = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]
print(n)
assert n == 0

0


In [917]:
# now we can combine both
df.loc[df["title_1"].isna(), "title_1"] = df["title_2"]
df = df.drop("title_2", axis=1)

In [918]:
# see what is left
df.loc[~df["title_1"].isna()].shape[0]

1819

In [919]:
df.loc[~df["title_1"].isna()].sample(5)

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2938,https://go.boarddocs.com/mi/hale/Board.nsf/Public,Hale Area Schools,http://www.haleschools.net,None,NaN,False,False,None
149,https://go.boarddocs.com/mi/hollan/Board.nsf/Public,Holland Public Schools,http://www.hollandpublicschools.org/,None,NaN,False,False,None
2130,https://go.boarddocs.com/la/acadia/Board.nsf/Public,Acadia Parish Schools,http://www.acadia.k12.la.us/,"2402 North Parkerson Avenue • PO Drawer 309 (70527-0309)| Crowley, LA 70526 | 337-783-3664 | f 337-783-3761",NaN,False,False,None
943,https://go.boarddocs.com/vsba/amelia/Board.nsf/Public,Amelia County Public Schools,http://www.amelia.k12.va.us/index.htm,"8701 Otterburn Road | Suite 101 | Amelia, Virginia 23002 | 804-561-2621",NaN,False,False,None
1185,https://go.boarddocs.com/in/nnsc/Board.nsf/Public,North Newton School Corporation,http://www.nn.k12.in.us,"310 S. Lincoln St., PO Box 8 | Morocco, IN 47963 | 219.285.2228 | f 219.285.2708",NaN,False,False,None


In [920]:
# check those with numbers
df.loc[df["title_1"].str.contains(number_pattern, na=False)].shape[0]

134

In [921]:
df.loc[df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
38,https://go.boarddocs.com/mo/mcr1/Board.nsf/Public,Macon County R-1 Schools,http://www.macon.k12.mo.us/,"702 North Missouri  Macon, Missouri 63552  (660) 395-6164",NaN,False,False,None
49,https://go.boarddocs.com/ks/susdks/Board.nsf/Public,Seaman USD 345,http://www.seamanschools.org,"901 NW Lyman Road | Topeka, KS 66608 | Ph: (785) 575-8600",NaN,False,False,None
76,https://go.boarddocs.com/pa/iu11/Board.nsf/Public,Tuscarora Intermediate Unit 11,https://www.tiu11.org/,None,NaN,False,False,None
125,https://go.boarddocs.com/wi/pesh/Board.nsf/Public,341 NORTH EMERY AVENUE,www.peshtigo.k12.wi.us,None,PESHTIGO SCHOOL DISTRICT,False,False,None
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
...,...,...,...,...,...,...,...,...
3806,https://go.boarddocs.com/ks/usd315/Board.nsf/Public,Colby Public Schools USD 315,http://www.colbyeagles.org/,600 W 3rd St. | Colby KS 67701-2000 | p 785-460-5000 | f 785-460-5050,NaN,False,False,None
3816,https://go.boarddocs.com/fl/semi/Board.nsf/Public,Phone - 407-320-0000,www.scps.k12.fl.us,"400 E. Lake Mary Boulevard - Sanford, FL - 32773",NaN,False,True,None
3820,https://go.boarddocs.com/ks/usd311/Board.nsf/Public,Pretty Prairie USD 311 KS,https://www.usd311.com/,"206 E Main, P.O. Box 218 Pretty Prairie, Kansas 67570",NaN,False,False,None
3877,https://go.boarddocs.com/pa/neiu/Board.nsf/Public,Northeastern Educational Intermediate Unit 19,http://www.iu19.org,1200 Line Street Archbald PA 18403,NaN,False,False,None


In [922]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]].shape[0]

38

In [923]:
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
680,https://go.boarddocs.com/pa/prtg/Board.nsf/Public,(814) 736-9636,www.portageareasd.org,"84 Mountain Avenue, Portage, PA 15946",NaN,True,True,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,(906) 224-7211,http://www.wmschools.org,None,Wakefield-Marenisco School District,True,True,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None


In [924]:
# move those without alphabets to the phone col

alphabet_pattern = r'[a-zA-Z]'
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)) ]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
680,https://go.boarddocs.com/pa/prtg/Board.nsf/Public,(814) 736-9636,www.portageareasd.org,"84 Mountain Avenue, Portage, PA 15946",NaN,True,True,None
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,(906) 224-7211,http://www.wmschools.org,None,Wakefield-Marenisco School District,True,True,None
1765,https://go.boarddocs.com/pa/iu14/Board.nsf/Public,610-987-2248,http://www.berksiu.org/,"1111 Commons Boulevard, PO Box 16050, 19612-6050",NaN,True,True,None
1924,https://go.boarddocs.com/wi/shorewood/Board.nsf/Public,(414) 963-6901,https://www.shorewood.k12.wi.us,"Shorewood School District | 1701 East Capitol Drive | Shorewood, Wisconsin 53211",NaN,True,True,None
1985,https://go.boarddocs.com/pa/roch/Board.nsf/Public,724-775-7500,http://www.rasd.org,"540 Reno Street, Rochester, PA 15074",NaN,True,True,None
2028,https://go.boarddocs.com/mi/fowler/Board.nsf/Public,(989)593-2250,http://www.fowlerschools.net,"700 S Main Street, Fowler MI 48835",NaN,True,True,None
2525,https://go.boarddocs.com/nj/burlingtontwp/Board.nsf/Public,(609) 387-3955,https://burltwpsch.org/,None,Burlington Township School District,True,True,None
2767,https://go.boarddocs.com/in/jcdc/Board.nsf/Public,812-689-4114,http://www.jaccendel.k12.in.us,"723 N Buckeye St, Osgood IN 47037",NaN,True,True,None
2874,https://go.boarddocs.com/wi/hfj1/Board.nsf/Public,(262) 673-3155,www.hjt1.org,"School District of Hartford Jt. #1, 402 W. Sumner St. Hartford, WI 53027",NaN,True,True,None
3159,https://go.boarddocs.com/wi/wawmsd/Board.nsf/Public,414-604-3000,http://www.wawmsd.org,"West Allis-West Milwaukee School District, 9333 W. Lincoln Avenue, West Allis, WI 53227",NaN,True,True,None


In [925]:
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)), "phone"] = df["title_1"]
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)), "title_1"] = None


In [926]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]].shape[0]

28

In [927]:
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
1574,https://go.boarddocs.com/oh/swissohio/Board.nsf/Public,Phone: 740-472-5801,https://swissohio.k12.oh.us,"304 Mill Street Woodsfield, OH 43793",NaN,True,True,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None


In [928]:
# the relevant info in the column is either phone, address or both
# the row with the website already has the website col filled
# first get those that are phone only

In [929]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*[Pp](hone)[^a-zA-Z]*$'

# Filter the rows
df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/2831296269.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1574,https://go.boarddocs.com/oh/swissohio/Board.nsf/Public,Phone: 740-472-5801,https://swissohio.k12.oh.us,"304 Mill Street Woodsfield, OH 43793",NaN,True,True,None
2140,https://go.boarddocs.com/wi/solon/Board.nsf/Public,Phone: (715) 378-2263,https://www.solonk12.net,"School District of Solon Springs, 8993 E Baldwin Ave, Solon Springs, WI 54873",NaN,True,True,None
2160,https://go.boarddocs.com/oh/galionoh/Board.nsf/Public,Phone - 419-468-3432,https://www.galionschools.org/,"Galion City Schools - 470 Portland Way North - Galion, OH 44833",NaN,True,True,None
3816,https://go.boarddocs.com/fl/semi/Board.nsf/Public,Phone - 407-320-0000,www.scps.k12.fl.us,"400 E. Lake Mary Boulevard - Sanford, FL - 32773",NaN,True,True,None


In [930]:
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/3918443988.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/3918443988.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None


In [931]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,True,True,None


In [932]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*([Pp]hone|PH|MAIN PHONE)?[^a-zA-Z]*([Ff]ax)?[^a-zA-Z]*$'

# Filter the rows
df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/3636070497.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
2538,https://go.boarddocs.com/wi/campsd/Board.nsf/Public,(920) 533-8381 | Fax (920) 533 -5726,www.csd.k12.wi.us,"327 N. Fond du Lac Ave. Campbellsport, WI 53010",NaN,True,True,None
2586,https://go.boarddocs.com/oh/cevsdoh/Board.nsf/Public,"Phone: 330-627-2181, - Fax: 330-627-2182",www.carrollton.k12.oh.us,"Carrollton Exempted Village School District, - 205 Scio Road S.W., - Carrollton, OH 44615",NaN,True,True,None
2811,https://go.boarddocs.com/id/nsd131/Board.nsf/Public,(208) 468-4600 Fax: (208) 468-4638,http://www.nsd131.org,"619 S. Canyon St | Nampa, ID 83686",NaN,True,True,None
3001,https://go.boarddocs.com/fl/highlfl/Board.nsf/Public,PH: 863-471-5555,www.highlands.k12.fl.us,"School Board of Highlands County, 426 School St., Sebring, FL 33870",NaN,True,True,None
3758,https://go.boarddocs.com/fl/brevco/Board.nsf/Public,* MAIN PHONE: (321) 633-1000,https://www.brevardschools.org/,"2700 JUDGE FRAN JAMIESON WAY, VIERA, FL 32940",NaN,True,True,None


In [933]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*([Pp]hone|PH|MAIN PHONE)?[^a-zA-Z]*([Ff]ax)?[^a-zA-Z]*$'

# Filter the rows
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/464474225.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/464474225.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None


In [934]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,True,True,None
2366,https://go.boarddocs.com/oh/naps/Board.nsf/Public,www.napoleonareaschools.org | 419-599-7015,www.napoleon.k12.oh.us,None,Napoleon Area City School District,True,False,None


In [935]:
# handle special cases
move_these = ["https://go.boarddocs.com/ks/usd230/Board.nsf/Public","https://go.boarddocs.com/oh/naps/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "phone"] = df["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

In [936]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2495,https://go.boarddocs.com/in/brem/Board.nsf/Public,Bremen Public Schools | Phone: (574) 546-3929 | Fax: (574) 546-6303 | School Board Policies and Guidelines,https://www.bps.k12.in.us,None,NaN,True,False,None
2808,https://go.boarddocs.com/wi/ashland/Board.nsf/Public,"District Office - 2000 Beaser Avenue; Ashland, WI; (715) 682-7080",http://www.ashland.k12.wi.us/,None,School District of Ashland Board of Education,True,False,None


In [937]:
# these are all addresses, move them
df.loc[df["contains_phone_number_title_1"], "address"] = df["title_1"]
df.loc[df["contains_phone_number_title_1"], "title_1"] = None

In [938]:
# now check the remaining ones
print(df[~df["title_1"].isna()].shape[0])
df[~df["title_1"].isna()].sample(5)

1781


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
412,https://go.boarddocs.com/oh/mcdonald/Board.nsf/Public,McDonald Local Schools,https://www.mcdonald.k12.oh.us/,"600 Iowa Avenue | McDonald, OH 44437 | 330-530-8051",NaN,False,False,None
3783,https://go.boarddocs.com/co/cde/Board.nsf/Public,Colorado Board of Education,http://www.cde.state.co.us,None,NaN,False,False,None
3248,https://go.boarddocs.com/mo/lathrop/Board.nsf/Public,Board of Education Room,http://www.lathropschools.com,None,Lathrop R-II School District,False,False,None
3463,https://go.boarddocs.com/nj/bectonhs/Board.nsf/Public,Henry P Becton Regional High School,https://www.bectonhs.org/,"120 Paterson Ave | East Rutherford, NJ 07073",NaN,False,False,None
576,https://go.boarddocs.com/nj/bpsnj/Board.nsf/Public,Bellmawr Board of Education eGovernance Site,http://www.bellmawrschools.org,"256 Anderson Ave | Bellmawr, NJ 08031 | 856-931-3620",NaN,False,False,None


In [939]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
901,https://go.boarddocs.com/nd/bsd7/Board.nsf/Public,"Agendas, Minutes, District Information, Policies and Events",http://www.belcourt.k12.nd.us/education/components/scrapbook/default.php?sectiondetailid=1376&,None,Turtle Mountain Community Schools/Belcourt School District #7,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
1736,https://go.boarddocs.com/pa/uncf/Board.nsf/Public,"Meetings, Agendas, and Information",http://www.ucfsd.org,None,Unionville-Chadds Ford School District,False,False,None
1800,https://go.boarddocs.com/pa/boyr/Board.nsf/Public,"Meetings, Agendas, Information",http://www.boyertownasd.org,None,NaN,False,False,None
2077,https://go.boarddocs.com/mo/unionrxi/Board.nsf/Public,"Meetings, Agendas, and Information",https://www.unionrxi.org,None,Union R-XI School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2521,https://go.boarddocs.com/oh/brightoh/Board.nsf/Public,"Meeting Agendas, Minutes and Policies",https://www.blsd.us,None,Bright Local School District,False,False,None


In [940]:
# the agenda value keep coming up
# let's check for all rows on this
df.loc[df["title_1"].str.contains("agenda",case=False,na=False)]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
157,https://go.boarddocs.com/il/sd163/Board.nsf/Public,Board Meeting Agendas,http://www.sd163.com,None,NaN,False,False,None
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,Laurel School Board Agendas and Policy Manual,http://www.laurel.k12.pa.us,None,Laurel School District,False,False,None
901,https://go.boarddocs.com/nd/bsd7/Board.nsf/Public,"Agendas, Minutes, District Information, Policies and Events",http://www.belcourt.k12.nd.us/education/components/scrapbook/default.php?sectiondetailid=1376&,None,Turtle Mountain Community Schools/Belcourt School District #7,False,False,None
1163,https://go.boarddocs.com/ca/vibrantminds/Board.nsf/Public,"Board Agendas, Minutes, and Policies",https://www.vibrantminds.us/,"412 W. CARL KARCHER WAY ANAHEIM, CA 92801 | ​714-563-2390",NaN,False,False,None
1736,https://go.boarddocs.com/pa/uncf/Board.nsf/Public,"Meetings, Agendas, and Information",http://www.ucfsd.org,None,Unionville-Chadds Ford School District,False,False,None
1800,https://go.boarddocs.com/pa/boyr/Board.nsf/Public,"Meetings, Agendas, Information",http://www.boyertownasd.org,None,NaN,False,False,None
2077,https://go.boarddocs.com/mo/unionrxi/Board.nsf/Public,"Meetings, Agendas, and Information",https://www.unionrxi.org,None,Union R-XI School District,False,False,None
2472,https://go.boarddocs.com/pa/iu24/Board.nsf/Public,Board Agendas and Policy Manual,http://www.cciu.org,"455 Boot Road, Downingtown, PA 19335 | 484-237-5000",NaN,False,False,None
2521,https://go.boarddocs.com/oh/brightoh/Board.nsf/Public,"Meeting Agendas, Minutes and Policies",https://www.blsd.us,None,Bright Local School District,False,False,None
2646,https://go.boarddocs.com/oh/mayoh/Board.nsf/Public,Meeting Agendas & Minutes,http://www.mayfieldschools.org,None,Mayfield City School District,False,False,None


In [941]:
# remove them

df.loc[df["title_1"].str.contains("agenda",case=False,na=False), "title_1"] = None

In [942]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2627,https://go.boarddocs.com/nj/hhboe/Board.nsf/Public,"316-A Seventh Ave Haddon Heights, New Jersey",http://gogarnets.com/,None,Haddon Heights School District,False,False,None
2710,https://go.boarddocs.com/ak/swrsdak/Board.nsf/Public,"... educating our future, guided by our past",http://www.swrsd.org,None,Southwest Region School District,False,False,None
2729,https://go.boarddocs.com/ca/cvesd/Board.nsf/Public,"84 East J Street , Chula Vista , CA91910",https://www.cvesd.org/,None,Chula Vista Elementary School District,False,False,None
2824,https://go.boarddocs.com/mo/wpr7sd/Board.nsf/Public,"Excellence in Education, Service, Life.",https://www.zizzers.org,None,West Plains School District,False,False,None
3287,https://go.boarddocs.com/wi/afasd/Board.nsf/Public,"201 W. 6th Street, Friendship, WI",https://www.afasd.net/,None,ADAMS-FRIENDSHIP AREA SCHOOL DISTRICT,False,False,None


In [943]:
# get those with numbers first

df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna() & df["title_1"].str.contains(number_pattern),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2627,https://go.boarddocs.com/nj/hhboe/Board.nsf/Public,"316-A Seventh Ave Haddon Heights, New Jersey",http://gogarnets.com/,None,Haddon Heights School District,False,False,None
2729,https://go.boarddocs.com/ca/cvesd/Board.nsf/Public,"84 East J Street , Chula Vista , CA91910",https://www.cvesd.org/,None,Chula Vista Elementary School District,False,False,None
3287,https://go.boarddocs.com/wi/afasd/Board.nsf/Public,"201 W. 6th Street, Friendship, WI",https://www.afasd.net/,None,ADAMS-FRIENDSHIP AREA SCHOOL DISTRICT,False,False,None


In [944]:
# move them to the address
idx = df["title_1"].str.contains(",",na=False) & df["address"].isna() & df["title_1"].str.contains(number_pattern)
df.loc[idx,"address"] = df["title_1"]
df.loc[idx,"title_1"] = None

In [945]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2710,https://go.boarddocs.com/ak/swrsdak/Board.nsf/Public,"... educating our future, guided by our past",http://www.swrsd.org,None,Southwest Region School District,False,False,None
2824,https://go.boarddocs.com/mo/wpr7sd/Board.nsf/Public,"Excellence in Education, Service, Life.",https://www.zizzers.org,None,West Plains School District,False,False,None


In [946]:
# move the first one to address and remove the rest, which are slogans
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "title_1"] = None

df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),"title_1"] = None

In [947]:
# check the remaining ones
print(df.loc[~df["title_1"].isna()].shape[0])
df.loc[~df["title_1"].isna()].sample(5)

1757


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1645,https://go.boarddocs.com/mi/beld/Board.nsf/Public,Belding Area Schools,http://www.bas-k12.org/,None,NaN,False,False,None
1691,https://go.boarddocs.com/oh/willard/Board.nsf/Public,Willard City Schools,www.willardschools.org,None,NaN,False,False,None
683,https://go.boarddocs.com/ny/btboces/Board.nsf/Public,Broome-Tioga BOCES,http://www.btboces.org/,"435 Glenwood Road | Binghamton, New York | 13905 | Ph: 607-766-3802 Fx: 607-763-3691",NaN,False,False,None
2894,https://go.boarddocs.com/oh/streetsboro/Board.nsf/Public,Streetsboro City Schools,http://www.scsrockets.org/,"9000 Kirby Lane | Streetsboro, OH 44241 | P 330-626-4900 | F 330-626-8102",NaN,False,False,None
1423,https://go.boarddocs.com/mo/kcpsrs/Board.nsf/Public,Kansas City Public School Retirement System,https://www.kcpsrs.org/,"3100 Broadway Street, Suite 1211, Kansas City, MO 64111",NaN,False,False,None


In [948]:
# check those with address na too
print(df.loc[~df["title_1"].isna() & df["address"].isna()].shape[0])
df.loc[~df["title_1"].isna() & df["address"].isna()].sample(5)

559


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
3083,https://go.boarddocs.com/oh/conne/Board.nsf/Public,Conneaut Area City Schools,http://www.cacsk12.org/,None,NaN,False,False,None
1610,https://go.boarddocs.com/oh/windev/Board.nsf/Public,Windham Exempted Village Schools,https://www.windham-schools.org/,None,NaN,False,False,None
2284,https://go.boarddocs.com/mi/mps/Board.nsf/Public,City of Muskegon Public Schools,https://muskegonpublicschools.org/,None,NaN,False,False,None
85,https://go.boarddocs.com/MI/Sparta/Board.nsf/Public,Sparta Area Schools,http://www.spartaschools.org,None,NaN,False,False,None
1172,https://go.boarddocs.com/butler/Board.nsf/Public,The Butler Technology and Career Development Schools,http://www.butlertech.org,None,NaN,False,False,None


In [949]:
# check those with school_district na too
print(df.loc[~df["title_1"].isna() & df["school_district"].isna()].shape[0])
df.loc[~df["title_1"].isna() & df["school_district"].isna()].sample(5)

1704


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1440,https://go.boarddocs.com/oh/jclocal/Board.nsf/Public,Jackson Center Local Schools,http://www.jctigers.org/Default.aspx,"204 S. Linden Street | Jackson Center, OH 45334 | 937-596-6053",NaN,False,False,None
381,https://go.boarddocs.com/la/tpss/Board.nsf/Public,Tangipahoa Parish School System,http://www.tangischools.org/,"59656 Puleston Road | Amite, LA 70422 | P: 985-748-7153 | F: 985-748-8587",NaN,False,False,None
3830,https://go.boarddocs.com/nj/flps/Board.nsf/Public,Fair Lawn Public Schools,https://www.fairlawnschools.org/,"37-01 Fair Lawn Ave. Fair Lawn, NJ 07410",NaN,False,False,None
485,https://go.boarddocs.com/in/nws/Board.nsf/Public,Northeastern Wayne School Corporation IN,https://www.nws.k12.in.us,"7299 US 27 N., P.O. Box 406 Fountain City, IN. 47341 | (765) 847-2821",NaN,False,False,None
2361,https://go.boarddocs.com/nj/htps/Board.nsf/Public,Hillsborough Township Public Schools,http://www.htps.us/,"379 South Branch Road | Hillsborough, NJ 08844 | (908) 431-6600",NaN,False,False,None


In [950]:
# check those with non-alphabets in title_1
non_alpha_pattern = r'[^A-Za-z ]'
print(df.loc[df["title_1"].str.contains(non_alpha_pattern, na=False)].shape[0])
df.loc[df["title_1"].str.contains(non_alpha_pattern, na=False)].sample(5)

319


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
3276,https://go.boarddocs.com/oh/cardingl/Board.nsf/Public,Cardington-Lincoln Local Schools,http://www.cardington.k12.oh.us,None,NaN,False,False,None
1599,https://go.boarddocs.com/in/trico/Board.nsf/Public,Tri-County School Corporation,http://www.trico.k12.in.us,None,NaN,False,False,None
3156,https://go.boarddocs.com/il/elmhurst/Board.nsf/Public,Elmhurst Community Unit School Dist. 205,http://www.elmhurst205.org,None,NaN,False,False,None
668,https://go.boarddocs.com/pa/iu09/Board.nsf/Public,Seneca Highlands Intermediate Unit 9,http://www.iu9.org,"119 S Mechanic St • Smethport, PA 16749",NaN,False,False,None
1476,https://go.boarddocs.com/la/stpsb/Board.nsf/Public,St. Tammany Parish Public School System,http://www.stpsb.org/,"321 N. Theard Street  Covington, LA 70433  985-892-2276  f 985-898-3267",NaN,False,False,None


In [951]:
# for those that have addresses set, we can move them to school_district
df.loc[~df["address"].isna() & ~df["title_1"].isna()]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,http://www.mapleschools.com,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",NaN,False,False,None
10,https://go.boarddocs.com/in/centergrove/Board.nsf/Public,Center Grove Community School Corporation,https://www.centergrove.k12.in.us/,"4800 West Stones Crossing Road | Greenwood, IN 46143 | (317) 881-9326",NaN,False,False,None
12,https://go.boarddocs.com/oh/pcc/Board.nsf/Public,Penta Career Center,http://www.pentacareercenter.org/,"9301 Buck Rd. | Perrysburg, Ohio 43551 | High School: 419-666-1120 Adult Education: 419-661-6555",NaN,False,False,None
15,https://go.boarddocs.com/oh/westholmes/Board.nsf/Public,West Holmes Local Schools,https://westholmes.org/,"28 W Jackson St. | Millersburg, OH 44654 | 330-674-3546",NaN,False,False,None
16,https://go.boarddocs.com/mabe/carps/Board.nsf/Public,Caroline County Public Schools,carolineschools.org,"Address: 204 Franklin Street | Denton, MD 21629 | Phone: (410) 479-1460",NaN,False,False,None
...,...,...,...,...,...,...,...,...
3897,https://go.boarddocs.com/co/jeffco/Board.nsf/Public,Jeffco Public Schools Board of Education,http://www.jeffcopublicschools.org/,1829 Denver West Drive | Golden. CO 80401 | (303) 982-6800,NaN,False,False,None
3902,https://go.boarddocs.com/il/cowil/Board.nsf/Public,City of Waukegan,http://www.waukeganil.gov/,"100 N. Martin Luther King Jr. Ave. | Waukegan, IL 60085",NaN,False,False,None
3903,https://go.boarddocs.com/ca/laccd/Board.nsf/Public,Los Angeles Community College District,http://laccd.edu,"770 Wilshire Boulevard, Los Angeles, CA 90017 | (213) 891-2000",NaN,False,False,None
3904,https://go.boarddocs.com/md/stmarysco/Board.nsf/Public,"St. Mary's County, Maryland",http://www.stmarysmd.com,"41770 Baldridge Street | Leonardtown, MD 20650 | 301-475-4200",NaN,False,False,None


In [952]:
# check if there are any where all three are set
df.loc[~df["address"].isna() & ~df["title_1"].isna() & ~df["school_district"].isna()]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone


In [953]:
# not really sure if it will be entirely correct
# check for those without address set
print(df.loc[df["address"].isna() & ~df["title_1"].isna()].shape[0])
df.loc[df["address"].isna() & ~df["title_1"].isna()].head()

559


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,St. Joseph Public Schools,https://www.sjschools.org/,None,NaN,False,False,None
6,https://go.boarddocs.com/pa/fcctc/Board.nsf/Public,Franklin County Career and Technology Center,www.franklinctc.com,None,NaN,False,False,None
9,https://go.boarddocs.com/ca/voc/Board.nsf/Public,Vista Oaks Charter School,https://www.vistaoaks.net,None,NaN,False,False,None
20,https://go.boarddocs.com/mi/man/Board.nsf/Public,Manton Consolidated Schools,http://mantonmi.apptegy.us/o/mcs?mode=edit,None,NaN,False,False,None
21,https://go.boarddocs.com/pa/cdsd/Board.nsf/Public,Quality to the Core,http://www.cdschools.org,None,Central Dauphin School District,False,False,None


In [954]:
# not really sure if it will be entirely correct
# check for those with both unset
print(df.loc[df["address"].isna() & df["school_district"].isna() & ~df["title_1"].isna()].shape[0])
df.loc[df["address"].isna() & df["school_district"].isna() & ~df["title_1"].isna()].sample(5)

506


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1146,https://go.boarddocs.com/ny/aplib/Board.nsf/Public,Albany Public Library NY,https://www.albanypubliclibrary.org/,None,NaN,False,False,None
2080,https://go.boarddocs.com/in/salm/Board.nsf/Public,Salem Community Schools,http://www.salemschools.com,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
2788,https://go.boarddocs.com/mi/fent/Board.nsf/Public,FENTON AREA PUBLIC SCHOOLS,https://www.fentonschools.org,None,NaN,False,False,None
3601,https://go.boarddocs.com/wi/bgf/Board.nsf/Public,BIG FOOT UNION HIGH SCHOOL,www.bigfoot.k12.wi.us,None,NaN,False,False,None


In [955]:
# check how many we have left
sum(~df["title_1"].isna())

1757

In [956]:
# send to GPT to deal with it
df.to_csv("test.csv", columns=["title_1","address","school_district"], index=False)

In [957]:
# I gave GPT-4o the following prompt
"""
This csv has three columns: title_1, address, and school_district. Many values are empty.

Your goal is to move the value from title_1 to either address, or school_district, or simply discard it. Do not overwrite any values.

For example, for the row

Christopher Newport University,"1 Avenue of the Arts Newport News, VA 23606 | (757) 594-7000",

You should put Christopher Newport University as the school_district, which is third column, that is

,"1 Avenue of the Arts Newport News, VA 23606 | (757) 594-7000","Christopher Newport University"

Another example

Quality to the Core,,Central Dauphin School District

You should just drop the title_1, since it is a slogan

,,Central Dauphin School District

Another example

75 Chenango Ave Clinton NY13323,,Clinton Central School District

You should put this in the address column

,75 Chenango Ave Clinton NY13323,Clinton Central School District

Return a csv
"""

# followed by 

"""
Ok. Now instead of moving, add a fourth column that says "address", "school_district", "None" or "drop"
"""

# it returns the following csv

'\nOk. Now instead of moving, add a fourth column that says "address", "school_district", "None" or "drop"\n'

In [958]:
gpt_df = pd.read_csv("classified_test_by_gpt.csv")
gpt_df.sample(10)

,title_1,address,school_district,classification
3782,Costa Mesa Sanitary District,"290 Paularino Avenue | Costa Mesa, CA 92626-3314 | (949) 645-8400 | f (714) 540-1392",NaN,school_district
1075,NaN,"8000 East Prairie Road Skokie, Illinois 60076",Skokie School District 73.5,NaN
2384,NaN,"533 Dayton Street, Box 627, Hamilton, OH 45011 | 513-887-5000",Hamilton City School District,NaN
3113,NaN,"1751 Earl L. Core Rd | Morgantown, WV 26505 | 304.291.9210",Monongalia County School District,NaN
3269,Ubly Community Schools,NaN,NaN,school_district
3205,Salmon River JSD 243 ID,"711 Ace's Place (PO Box 872) Riggins, ID 83549 | 208.630.6027",NaN,school_district
1832,Randolph Central School Corporation,NaN,NaN,school_district
1823,International Charter School of Atlanta,"1335 Northmeadow Parkway Suite 100, Roswell, GA 30076, US",NaN,school_district
476,NaN,NaN,City of St. Charles R-VI School District,NaN
2976,NaN,"1402 E. Janss Rd. | Thousand Oaks, CA 91362 | (805) 497-9511",Conejo Valley Unified School District,NaN


In [959]:
# let's check its distribution
gpt_df["classification"].value_counts(dropna=False)

classification
NaN                2149
school_district    1704
address              53
Name: count, dtype: int64

In [960]:
# interestingly, there are no drops
# let's look at the slogan row
gpt_df[gpt_df["title_1"]=="Quality to the Core"]

,title_1,address,school_district,classification
21,Quality to the Core,NaN,Central Dauphin School District,address


In [961]:
# this is a pretty bad classification
# turns out GPT was using Python under the hood
# I asked GPT to classify manually instead
# here is its results
gpt_df = pd.read_csv("manual_classified_test.csv")
gpt_df.sample(10)

,title_1,address,school_district,classification
3047,Morrice Area Schools,NaN,NaN,school_district
3480,NaN,"Fremont Community Schools | 1100 West Toledo Street | Fremont, IN 46737 | 260-495-5005",NaN,NaN
1581,NaN,"200 PHILLIPS DRIVE, SAUSALITO (MARIN CITY), CA 94965 | 415-332-3190",Sausalito Marin City School District,NaN
637,"City of Elmhurst, IL","209 N. York St | Elmhurst, IL 60126 | P (630) 530-3000",NaN,school_district
2690,Ladue Schools,"9703 Conway Road | St. Louis, MO 63124-1646 | Ph: 314.994.7080 Fx: 314.994.0441",NaN,school_district
1182,Seneca East Local Schools,"13343 E. US Highway 224, Attica, OH 44807",NaN,school_district
2727,NaN,NaN,NaN,NaN
1015,Turtle Mountain Community College,"10145 BIA Road 7 | PO Box 340 | Belcourt, ND 58316 | P 701.369.6060",NaN,school_district
87,NaN,"301 Cayucos Drive, Cayucos, CA 93430 | (805) 995-3694",Cayucos Elementary School District,NaN
3256,Fall River Public Schools,"417 Rock Street Fall River, MA 02720 | 508.675.8420",NaN,school_district


In [962]:
# still terrible, and I realized it still uses python under the hood
# I will start a new session and emphasize not to program
# let's check a few stats first

# number of NA title_1
print(df[df["title_1"].isna()].shape[0])

# number of none NA title_1
print(df[~df["title_1"].isna()].shape[0])

2149
1757


In [963]:
df.columns

Index(['URL', 'title_1', 'home_website', 'address', 'school_district',
       'contains_phone_number_title_1', 'contains_phone_number_title_2',
       'phone'],
      dtype='object')

In [964]:
df = df.drop(columns=["contains_phone_number_title_1","contains_phone_number_title_2"])

In [965]:
# try again
df["index"] = df.index

In [966]:
# o1 identified these as addresses
print(df.loc[df["index"].isin([155,242, 1343])])
# handle them manually
df.loc[df["URL"]=="https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public", "title_1"] = None

# drop the rest
df.loc[df["index"].isin([242, 1343]), "title_1"] = None

                                                          URL  \
155   https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public   
242          https://go.boarddocs.com/oh/swl/Board.nsf/Public   
1343        https://go.boarddocs.com/pa/karn/Board.nsf/Public   

                              title_1               home_website  \
155   75 Chenango Ave Clinton NY13323       https://www.ccs.edu/   
242          Southwest Licking LSD OH   http://www.swl.k12.oh.us   
1343             1446 Kittanning Pike  https://www.kcasdk12.org/   

                                                                       address  \
155                                                                       None   
242   Address: 927-A South Street, Pataskala, Ohio 43062 | Phone: 740-927-3941   
1343                                                     Karns City, PA  16041   

                      school_district phone  index  
155   Clinton Central School District  None    155  
242                            

In [967]:
# output the rest
# use prompts/prompt_3.txt
df[~df["title_1"].isna()].to_csv("test.csv", columns=["title_1","index"],index=False)

In [968]:
# get the o1 classification
gpt_df = pd.read_csv("gpt_results/results_10.csv")

# merge them
df = pd.merge(df, gpt_df, how="left", on="index")


In [969]:
# get the missing ones
df[(~df["title_1"].isna()) & (df["classification"].isna()) ].to_csv("test-2.csv", columns=["title_1","index"],index=False)

In [970]:
# sanity check
# all NAs should not have classifications
assert df[(df["title_1"].isna()) & (~df["classification"].isna()) ].shape[0] == 0

# all non-NAs should have classifications
assert df[(~df["title_1"].isna()) & (df["classification"].isna()) ].shape[0] == 0

# the number of classification should be the number of rows on the right
assert gpt_df.shape[0] == df[~df["classification"].isna()].shape[0]

In [971]:
df["classification"].value_counts()

classification
school_district    1617
drop                 86
unsure               51
Name: count, dtype: int64

In [972]:
# check the ones that are unsure
df[df["classification"]=="unsure"]

,URL,title_1,home_website,address,school_district,phone,index,classification
26,https://go.boarddocs.com/sd/meade/Board.nsf/Public,"Meade County, South Dakota",http://www.meadecounty.org/,"1300 Sherman Street Ste 222 | Sturgis, SD 57785",NaN,None,26,unsure
152,https://go.boarddocs.com/ks/kta/Board.nsf/Public,Kansas Turnpike Authority,http://www.ksturnpike.com/,"9401 E Kellogg | Wichita, KS 67207 | (316) 682-4537",NaN,None,152,unsure
165,https://go.boarddocs.com/il/vobil/Board.nsf/Public,"The Village of Bourbonnais, Illinois",http://www.villageofbourbonnais.com/,"600 Main St N.W. | Bourbonnais, IL 60914",NaN,None,165,unsure
269,https://go.boarddocs.com/il/naperville/Board.nsf/Public,Naperville Park District,https://www.napervilleparks.org/,"320. W. Jackson Ave. Naperville, IL 60540 | 630-848-5000",NaN,None,269,unsure
303,https://go.boarddocs.com/mn/mboa/Board.nsf/Public,Minnesota Board on Aging,http://www.mn.gov/board-on-aging,"540 Cedar Street St. Paul, MN 55155 | 1-800-882-6262",NaN,None,303,unsure
320,https://go.boarddocs.com/ca/phcd/Board.nsf/Public,Peninsula Health Care District,http://www.peninsulahealthcaredistrict.org,"1819 Trousdale Drive | Burlingame, CA 94010 | (650) 697-6900",NaN,None,320,unsure
609,https://go.boarddocs.com/il/ctpf/Board.nsf/Public,Public School Teachers' Pension and Retirement Fund of Chicago,http://www.ctpf.org,"425 S Financial Place, Suite 1400, Chicago, IL 60605 | 312-604-1400",NaN,None,609,unsure
620,https://go.boarddocs.com/mi/jackson/Board.nsf/Public,"Jackson County, Michigan",http://www.mijackson.org,"120 West Michigan | Jackson, MI 49201 | P: (517) 788-4335 | F: (517) 780-4755",NaN,None,620,unsure
653,https://go.boarddocs.com/fl/sefbhn/Board.nsf/Public,Southeast Florida Behavioral Health Network,http://sefbhn.org/,"8895 N Military Trail, Suite E-102 | Palm Beach Gardens, FL 33410 | (561) 203-2485",NaN,None,653,unsure
697,https://go.boarddocs.com/ok/plib/Board.nsf/Public,Pioneer Library System,http://www.pioneerlibrarysystem.org,"300 Norman Center Court | Norman, OK 73072 | 405.801.4500",NaN,None,697,unsure


In [973]:
# the unsures all look like valid entities
# and their school_district field is empty
# we will classify them as school_district

df.loc[df["classification"]=="unsure", "classification"] = "school_district"

In [974]:
# look at those with drops
df.loc[df["classification"]=="drop",:].sample(5)

,URL,title_1,home_website,address,school_district,phone,index,classification
2443,https://go.boarddocs.com/ny/hilton/Board.nsf/Public,Home Page,https://www.hilton.k12.ny.us/,None,Hilton Central School District,None,2443,drop
226,https://go.boarddocs.com/wi/hgsd/Board.nsf/Public,Home of the Tigers,https://www.hgsd.k12.wi.us,"403 Audubon Road Howards Grove, WI 53083",NaN,None,226,drop
311,https://go.boarddocs.com/mi/huronvalley/Board.nsf/Public,"Inspiring and Building Futures, One Student at a Time",https://www.hvs.org,"2390 S Milford Road, Highland, MI 48357",NaN,None,311,drop
136,https://go.boarddocs.com/pa/iroq/Board.nsf/Public,Board Meeting Documents and Policies,http://www.iroquoissd.org,None,IROQUOIS SCHOOL DISTRICT,None,136,drop
3711,https://go.boarddocs.com/wi/sdwab/Board.nsf/Public,Wabeno School Board Policies,www.wabeno.k12.wi.us,None,NaN,None,3711,drop


In [975]:
# drop the ones classified as drops
df.loc[df["classification"]=="drop", "title_1"] = None
df.loc[df["classification"]=="drop", "classification"] = None

In [976]:
# look at those classified as school_district
df.loc[df["classification"]=="school_district",:].sample(5)

,URL,title_1,home_website,address,school_district,phone,index,classification
999,https://go.boarddocs.com/in/nisec/Board.nsf/Public,Northwest Indiana Special Education Cooperative,http://www.nisec.org,None,NaN,None,999,school_district
663,https://go.boarddocs.com/ks/pusd/Board.nsf/Public,Piper USD 203,https://www.piperschools.com/,"Piper District Office | 3130 N 122nd St. Suite A | Kansas City, KS 66109 | Ph: (913) 721-2088 | Fx: (913) 721-3573",NaN,None,663,school_district
3649,https://go.boarddocs.com/nc/surrync/Board.nsf/Public,Surry County Schools,https://www.surry.k12.nc.us,"PO Box 364, 209 N. Crutchfield Street Dobson, NC 27017 | 336-386-8211",NaN,None,3649,school_district
761,https://go.boarddocs.com/oh/gahanna/Board.nsf/Public,Gahanna-Jefferson Public Schools,http://www.gahannaschools.org/,"630 Morrison Road, Suite 200 Gahanna, OH 43230",NaN,None,761,school_district
1933,https://go.boarddocs.com/nj/ewing/Board.nsf/Public,Ewing Township Public Schools,http://www.ewing.k12.nj.us/,"2099 Pennington Road | Ewing, NJ 08618 | p (609) 538-9800 | f (609) 538-0041",NaN,None,1933,school_district


In [977]:
# move the others to school_district
df.loc[df["classification"]=="school_district", "school_district"] = df["title_1"]
df.loc[df["classification"]=="school_district", "title_1"] = None
df.loc[df["classification"]=="school_district", "classification"] = None

# assert no others
assert df[~df["classification"].isna()].shape[0] == 0
assert df[~df["title_1"].isna()].shape[0] == 0

In [978]:
# delete these columns
df = df.drop(["classification","title_1"], axis=1)

In [979]:
# we also try to get the phone number from the address
df['address_contains_phone_number'] = df['address'].apply(contains_phone_number)
df[df["address_contains_phone_number"]].sample(5)

,URL,home_website,address,school_district,phone,index,address_contains_phone_number
2962,https://go.boarddocs.com/nj/haledon/Board.nsf/Public,http://www.haledon.org/,"91 Henry Street | Haledon, NJ 07508 | (973)790-9000",Haledon Public School,None,2962,True
2330,https://go.boarddocs.com/wi/shelllake/Board.nsf/Public,http://www.shelllake.k12.wi.us/,"271 Hwy 63 | Shell Lake, WI 54871 | 715-468-7816 | f 715-468-7812",The School District of Shell Lake,None,2330,True
3827,https://go.boarddocs.com/ny/mechanicville/Board.nsf/Public,https://www.mechanicville.org,"25 Kniskern Avenue, Mechanicville, NY 12118 | (518) 664-5727",Mechanicville City School District,None,3827,True
1695,https://go.boarddocs.com/az/pusd4/Board.nsf/Public,https://www.pusdatsa.org,"1 Mile North of Hwy 41 Pinon, AZ 86510 | 928-725-3450",Pinon Unified School District,None,1695,True
97,https://go.boarddocs.com/nj/rvrhs/Board.nsf/Public,http://www.rvrhs.com/,"520 Jacksonville Road | Mt. Holly, NJ 08060 | (609) 267-0830",Rancocas Valley Regional High School,None,97,True


In [980]:
df[df["address_contains_phone_number"]].shape

(2185, 7)

In [981]:
# extract it
# but before that, check if conflicts
df[df["address_contains_phone_number"] & (~df["phone"].isna())].shape

(0, 7)

In [982]:
# great! no conflicts
phone_pattern = re.compile(r'(\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b)')
df["phone_extracted_from_address"] = df["address"].str.extract(phone_pattern)

In [983]:
df[~df["phone_extracted_from_address"].isna()].sample(5)

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address
3634,https://go.boarddocs.com/oh/wickliffe/Board.nsf/Public,http://www.cityofwickliffe.com,"28730 Ridge Road Wickliffe, OH 44092 | (440) 943-7100","City of Wickliffe, Ohio",None,3634,True,(440) 943-7100
2881,https://go.boarddocs.com/mi/lwacademy/Board.nsf/Public,https://www.lightoftheworldacademy.org/,"550 East Hamburg Street, Pinckney, MI 48169 Phone: (734) 720-9760",Light of the World Academy,None,2881,True,(734) 720-9760
2821,https://go.boarddocs.com/oh/whcs/Board.nsf/Public,http://www.warrensville.k12.oh.us,"4743 Richmond Road, Warrensville Hts., Ohio 44128 Phone: (216) 364-1001 Fax: (216) 364-1024",Warrensville Heights City Schools,None,2821,True,(216) 364-1001
1009,https://go.boarddocs.com/ks/usd469/Board.nsf/Public,http://www.usd469.net,"200 E. Mary| Lansing, KS 66043 | Ph: (913) 727-1100 | Fx: (913) 727-1619",Lansing USD 469,None,1009,True,(913) 727-1100
5,https://go.boarddocs.com/de/sussexvt/Board.nsf/Public,http://www.sussexvt.org,17099 County Seat Hwy | Georgetown DE 19947 | 302-856-0961,Sussex Technical School District,None,5,True,302-856-0961


In [984]:
# double check no clashes
df[(~df["phone_extracted_from_address"].isna()) & (~df["phone"].isna())]

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address


In [985]:
# write in
df.loc[~df["phone_extracted_from_address"].isna(), "phone"] = df["phone_extracted_from_address"]
df.loc[~df["phone_extracted_from_address"].isna(), "phone_extracted_from_address"] = None

In [986]:
# let's clean it
# extract all numbers and make sure there are 10

df['extracted_phone_numbers'] = None
df['extracted_phone_numbers'] = df['phone'].astype(str).str.replace(r'[^\d]', '', regex=True)

In [987]:
# check their length
df["phone_length"] = df["extracted_phone_numbers"].apply(lambda x: len(x))
df["phone_length"].value_counts()

phone_length
10    2216
0     1684
20       4
12       1
30       1
Name: count, dtype: int64

In [988]:
# check those that are not 10 or 0
df.loc[(df["phone_length"]!=10) & (df["phone_length"]!=0)]

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address,extracted_phone_numbers,phone_length
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,http://www.triton.k12.in.us,None,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,624,False,NaN,574342225512,12
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,Phone: 715-257-7511 Fax: 715-257-7502,1199,False,NaN,71525775117152577502,20
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,2173,False,NaN,913592727291359272009135927270,30
2538,https://go.boarddocs.com/wi/campsd/Board.nsf/Public,www.csd.k12.wi.us,"327 N. Fond du Lac Ave. Campbellsport, WI 53010",NaN,(920) 533-8381 | Fax (920) 533 -5726,2538,False,NaN,92053383819205335726,20
2586,https://go.boarddocs.com/oh/cevsdoh/Board.nsf/Public,www.carrollton.k12.oh.us,"Carrollton Exempted Village School District, - 205 Scio Road S.W., - Carrollton, OH 44615",NaN,"Phone: 330-627-2181, - Fax: 330-627-2182",2586,False,NaN,33062721813306272182,20
2811,https://go.boarddocs.com/id/nsd131/Board.nsf/Public,http://www.nsd131.org,"619 S. Canyon St | Nampa, ID 83686",NaN,(208) 468-4600 Fax: (208) 468-4638,2811,False,NaN,20846846002084684638,20


In [989]:
# ok, we can just take the top 10 numbers
df["extracted_phone_numbers"] = df["extracted_phone_numbers"].apply(lambda x: x[:10] if x else None)

In [990]:
# put them in a nice format
def format_phone_numbers(number):
    if not number:
        return None
    # Convert number to string for easy slicing
    num_str = str(number)
    if len(num_str) == 10:  # Ensure it's a 10-digit number
        formatted = f"({num_str[:3]}) {num_str[3:6]}-{num_str[6:]}"
        return formatted
    else:
        raise Exception("Invalid number")

df["phone"] = df["extracted_phone_numbers"].apply(format_phone_numbers)

In [991]:
# clean up the columns
df = df.loc[:,["URL","home_website","address","school_district","phone"]]
df.sample(5)

,URL,home_website,address,school_district,phone
2136,https://go.boarddocs.com/mi/eatonacad/Board.nsf/Public,https://www.eaton-academy.com/,"21450 Universal Ave., Eastpointe, MI 48021 Phone (586) 777-1519",Eaton Academy,(586) 777-1519
3286,https://go.boarddocs.com/pa/reyn/Board.nsf/Public,http://www.reynolds.k12.pa.us,"531 Reynolds Road Greenville, PA 16125",Reynolds School District,None
3074,https://go.boarddocs.com/az/lesdaz/Board.nsf/Public,http://www.lesd79.org,"272 East Sagebrush Street | Litchfield Park, AZ 85340 | (623) 535-6000",Litchfield Elementary School District #79,(623) 535-6000
200,https://go.boarddocs.com/oh/beaverls/Board.nsf/Public,http://www.beaver.k12.oh.us,None,Beaver Local Schools,None
899,https://go.boarddocs.com/wi/mtsd/Board.nsf/Public,http://www.mtsd.k12.wi.us/,"5000 W. Mequon Road | Mequon, WI 53092 | 262-238-8500",Mequon-Thiensville School District,(262) 238-8500


In [992]:
# finally, add a column indicate if the row is likely an actual school_district
# here is Tom's quote
"""
However, please add a column to the spreadsheet noting whether the words “school”, “academy”, “district” or “education” appear in the “address” or “school_district” columns.   That way, we can go through later and weed out the other organizations.
"""

'\nHowever, please add a column to the spreadsheet noting whether the words “school”, “academy”, “district” or “education” appear in the “address” or “school_district” columns.   That way, we can go through later and weed out the other organizations.\n'

In [993]:
likely_school_district_pattern = r"school|academy|district|education"
df["contain_school_district_keywords"] = df["address"].str.contains(likely_school_district_pattern, case=False, regex=True, na=False) | df["school_district"].str.contains(likely_school_district_pattern, case=False, regex=True, na=False) 

In [994]:
import validators

df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df["valid_home_website"].value_counts()

valid_home_website
True     3792
False     114
Name: count, dtype: int64

In [995]:
# non-null but invalid home website
weird_websites = df[(~df["valid_home_website"]) & (~df["home_website"].isna())]
weird_websites.shape

(61, 7)

In [996]:
weird_websites

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,https://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
173,https://go.boarddocs.com/in/wesdel/Board.nsf/Public,http://www.wes-del.k12.in us,None,Wes-Del Community Schools,None,True,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,https://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
...,...,...,...,...,...,...,...
3679,https://go.boarddocs.com/pa/iu08/Board.nsf/Public,http://www.iu08.org,None,Appalachia Intermediate Unit 8,None,False,False
3741,https://go.boarddocs.com/mi/wpcs/Board.nsf/Public,http:/www.wpcschools.org,None,White Pigeon Community Schools,None,True,False
3832,https://go.boarddocs.com/pa/fleb/Board.nsf/Public,http://www.fortleboeuf.net,None,Fort LeBoeuf School District,None,True,False
3876,https://go.boarddocs.com/pa/avnw/Board.nsf/Public,http://www.avonworth.k12.pa.us/,"258 Josephs Lane Pittsburgh, PA 15237",Avonworth School District,None,True,False


In [997]:
# replace \\
idx = (~df["valid_home_website"]) & (~df["home_website"].isna())
df.loc[idx,"home_website"] = df.loc[:,"home_website"].str.replace("\\","/")

In [998]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,https://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
173,https://go.boarddocs.com/in/wesdel/Board.nsf/Public,http://www.wes-del.k12.in us,None,Wes-Del Community Schools,None,True,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,https://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
434,https://go.boarddocs.com/wi/ncraw/Board.nsf/Public,www.northcrawford.com,"47050 County Road X | Soldiers Grove, WI 54655 | (608) 735-4318",North Crawford School District,(608) 735-4318,True,False
473,https://go.boarddocs.com/pa/bldw/Board.nsf/Public,http://www.bwschools.net/,None,Baldwin-Whitehall School District,None,True,False
569,https://go.boarddocs.com/wi/fond/Board.nsf/Public,http:/www.fonddulac.k12.wi.us,"72 West Ninth Street, Fond du Lac, WI 54935 (920) 906-6500",Fond du Lac School District,(920) 906-6500,True,False
611,https://go.boarddocs.com/pa/bgbf/Board.nsf/Public,http://www.tigerweb.org/,None,Big Beaver Falls School District,None,True,False
639,https://go.boarddocs.com/oh/young/Board.nsf/Public,http://www.ycsd.org,None,Youngstown City Schools District,None,True,False


In [999]:
# fix the typos
pattern = r"https?:?/?/(?=[A-Za-z])"
idx = (~df["valid_home_website"]) & (~df["home_website"].isna())
df.loc[idx,"home_website"] = df.loc[:,"home_website"].str.replace(pattern,"http://", regex=True)

In [1000]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,http://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
173,https://go.boarddocs.com/in/wesdel/Board.nsf/Public,http://www.wes-del.k12.in us,None,Wes-Del Community Schools,None,True,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,http://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
434,https://go.boarddocs.com/wi/ncraw/Board.nsf/Public,www.northcrawford.com,"47050 County Road X | Soldiers Grove, WI 54655 | (608) 735-4318",North Crawford School District,(608) 735-4318,True,False
473,https://go.boarddocs.com/pa/bldw/Board.nsf/Public,http://www.bwschools.net/,None,Baldwin-Whitehall School District,None,True,False
611,https://go.boarddocs.com/pa/bgbf/Board.nsf/Public,http://www.tigerweb.org/,None,Big Beaver Falls School District,None,True,False
639,https://go.boarddocs.com/oh/young/Board.nsf/Public,http://www.ycsd.org,None,Youngstown City Schools District,None,True,False
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,http://www.laurel.k12.pa.us,None,Laurel School District,None,True,False


In [1001]:
# fix one
df.loc[df["URL"]=="https://go.boarddocs.com/in/wesdel/Board.nsf/Public", "home_website"] = "http://www.wes-del.k12.in.us"

# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,http://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,http://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
434,https://go.boarddocs.com/wi/ncraw/Board.nsf/Public,www.northcrawford.com,"47050 County Road X | Soldiers Grove, WI 54655 | (608) 735-4318",North Crawford School District,(608) 735-4318,True,False
473,https://go.boarddocs.com/pa/bldw/Board.nsf/Public,http://www.bwschools.net/,None,Baldwin-Whitehall School District,None,True,False
611,https://go.boarddocs.com/pa/bgbf/Board.nsf/Public,http://www.tigerweb.org/,None,Big Beaver Falls School District,None,True,False
639,https://go.boarddocs.com/oh/young/Board.nsf/Public,http://www.ycsd.org,None,Youngstown City Schools District,None,True,False
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,http://www.laurel.k12.pa.us,None,Laurel School District,None,True,False
694,https://go.boarddocs.com/pa/eriesd/Board.nsf/Public,http://www.boarddocs.com/pa/eriesd/Board.nsf/Private?open&login,"148 West 21st Street | Erie, Pennsylvania 16502 | 814-874-6000","The School District of the City of Erie, Pennsylvania",(814) 874-6000,True,False


In [1002]:
# we don't care about the rest of the URL after the first singular /
# and we also don't care about spaces
# their spaces should be after / though

df["spaces_before_slash"] = df["home_website"].apply(lambda x: ' ' in str(x).replace("//","").split('/', 1)[0] if x else False)
df["spaces_before_slash"].value_counts()

spaces_before_slash
False    3884
True       22
Name: count, dtype: int64

In [1003]:
# chop off the rest of the URL if they have spaces
df.loc[:,"home_website"] = df["home_website"].apply(lambda x: str(x).split(' ', 1)[0] if x else None)

# also just keep the things before the slash
single_slash = r"(?<!/)/(?!/)"
df.loc[:,"home_website"] = df["home_website"].apply(lambda x: re.split(single_slash, str(x))[0] if x else None)

In [1004]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website,spaces_before_slash
232,https://go.boarddocs.com/pa/camb/Board.nsf/Public,nan,None,NaN,None,False,False,False
443,https://go.boarddocs.com/oh/oakhil/Board.nsf/Public,nan,None,NaN,None,False,False,False
465,https://go.boarddocs.com/oh/warrenoh/Board.nsf/Public,nan,None,NaN,None,False,False,False
468,https://go.boarddocs.com/pa/marp/Board.nsf/Public,nan,None,Marple Newtown School District,None,True,False,False
533,https://go.boarddocs.com/oh/meigs/Board.nsf/Public,nan,None,NaN,None,False,False,False
552,https://go.boarddocs.com/pa/sola/Board.nsf/Public,nan,None,NaN,None,False,False,False
590,https://go.boarddocs.com/in/lanesville/Board.nsf/Public,nan,None,Lanesville Comunity School Corporation,None,True,False,False
784,https://go.boarddocs.com/wi/sbschools/Board.nsf/Public,nan,None,Stanley-Boyd Area Schools,None,True,False,False
884,https://go.boarddocs.com/oh/vwcs/Board.nsf/Public,nan,None,Van Wert City Schools,None,True,False,False
996,https://go.boarddocs.com/ca/lmsv/Board.nsf/Public,nan,None,NaN,None,False,False,False


In [1005]:
df.loc[df["home_website"] == "nan", "home_website"] = None

In [1006]:
# fix manual
df.loc[df["URL"]=="https://go.boarddocs.com/pa/ojrsd/Board.nsf/Public", "home_website"] = "http://www.ojrsd.com"
df.loc[df["URL"]=="https://go.boarddocs.com/mi/sutt/Board.nsf/Public", "home_website"] = "http://www.suttonsbayschools.com"

In [1007]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website,spaces_before_slash


In [1008]:
# add http
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True)
df.loc[~df["valid_home_website"], "home_website"] = "http://" + df["home_website"]

In [1009]:
# non-null but invalid home website
df["valid_URL"] = df["URL"].apply(lambda x: validators.url(x) == True)
df["valid_URL"].value_counts()

valid_URL
True     3905
False       1
Name: count, dtype: int64

In [1010]:
df[~df["valid_URL"]]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website,spaces_before_slash,valid_URL
3553,NaN,NaN,None,NaN,None,False,False,False,False


In [1011]:
idx = df[~df["valid_URL"]].index
df = df.drop(idx,axis=0)

In [1012]:
# non-null but invalid home website
df["valid_URL"] = df["URL"].apply(lambda x: validators.url(x) == True)
df["valid_URL"].value_counts()

valid_URL
True    3905
Name: count, dtype: int64

In [1013]:
# check those that link to boarddocs
df["boarddocs_as_home_website"] = df["home_website"].str.contains("boarddocs", na=False)
df["boarddocs_as_home_website"].value_counts()

boarddocs_as_home_website
False    3873
True       32
Name: count, dtype: int64

In [1014]:
# just remove these
df.loc[df["boarddocs_as_home_website"], "home_website"] = None

In [1015]:
# output the result
df.to_csv("release/deliverable_1.csv", index=False, columns=["URL","school_district","address","home_website","phone"])

In [1016]:
# get the stats
df["contain_school_district_keywords"].value_counts()

contain_school_district_keywords
True     3407
False     498
Name: count, dtype: int64

In [1017]:
# trim all whitespace before output
df["URL"] = df["URL"].str.strip()
df["school_district"] = df["school_district"].str.strip()
df["address"] = df["address"].str.strip()
df["home_website"] = df["home_website"].str.strip()
df["phone"] = df["phone"].str.strip()

In [1018]:
# get more stats
df.shape

(3905, 10)

In [1019]:
len(df["URL"].unique())

3905

In [1020]:
# home website
(~df["home_website"].isna()).value_counts()

home_website
True     3820
False      85
Name: count, dtype: int64

In [1021]:
# home website (in percentages)
(~df["home_website"].isna()).value_counts() / df.shape[0] * 100

home_website
True     97.823303
False     2.176697
Name: count, dtype: float64

In [1022]:
# phone numbers
(~df["phone"].isna()).value_counts()

phone
True     2222
False    1683
Name: count, dtype: int64

In [1023]:
# phone (in percentages)
(~df["phone"].isna()).value_counts() / df.shape[0] * 100

phone
True     56.901408
False    43.098592
Name: count, dtype: float64

In [1024]:
# phone or website
((~df["phone"].isna()) | (~df["home_website"].isna())).value_counts()

True     3844
False      61
Name: count, dtype: int64

In [1025]:
# phone or website (percentages)
((~df["phone"].isna()) | (~df["home_website"].isna())).value_counts()/ df.shape[0] * 100

True     98.4379
False     1.5621
Name: count, dtype: float64

In [1026]:
# school_district
(~df["school_district"].isna()).value_counts()

school_district
True     3641
False     264
Name: count, dtype: int64

In [1027]:
# school_district (in percentages)
(~df["school_district"].isna()).value_counts() / df.shape[0] * 100

school_district
True     93.239437
False     6.760563
Name: count, dtype: float64

In [1028]:
# address
(~df["address"].isna()).value_counts()

address
True     2919
False     986
Name: count, dtype: int64

In [1029]:
# address (in percentages)
(~df["address"].isna()).value_counts() / df.shape[0] * 100

address
True     74.75032
False    25.24968
Name: count, dtype: float64

In [1030]:
# get the stats
df["contain_school_district_keywords"].value_counts()

contain_school_district_keywords
True     3407
False     498
Name: count, dtype: int64

In [1031]:
# address (in percentages)
df["contain_school_district_keywords"].value_counts() / df.shape[0] * 100

contain_school_district_keywords
True     87.247119
False    12.752881
Name: count, dtype: float64